# **Github Links**

**SqueezeDet**
https://github.com/BichenWuUCB/squeezeDet

**RefinedetLite**
https://github.com/ouyanghuiyu/RefinedetLite.pytorch

**ssdlite mobilenet v2**
https://github.com/qfgaohao/pytorch-ssd

https://github.com/tranleanh/MobileNets-SSD-PyTorch

https://github.com/shaoshengsong/MobileNetV3-SSD

**Controllable Invariance**
https://github.com/qizhex/Controllable-Invariance

* While Yolo has fixed grid cell aspect ratio. SSD uses different aspect ratio with multi boxes for better accuracy.

* SSD has additional conv layers at the end of the base mobilenet for object detection. convolutional layer has multiple features with different scale and hence it is able to detect objects in multiple scales better


Folder: https://drive.google.com/drive/u/1/folders/1trRWtHV_io4EDScepki1Q1RS_YewKdTA

In [1]:
import pickle
import logging
from datetime import datetime
import progressbar
from time import sleep

from six.moves import xrange
import threading

import gc 
import torch
import torch.nn as nn
from torch import cuda
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as torchdata
from torch.autograd import Variable
from torchsummary import summary
from torch.utils.data import DataLoader, ConcatDataset
from torch.optim.lr_scheduler import CosineAnnealingLR, MultiStepLR
import xml.etree.ElementTree as ET

import collections
from collections import deque
from typing import List, Tuple
import math
import time
import os
import os.path
import sys
import random
import copy
import cv2
import itertools
import pathlib
import numpy as np
import pandas as pd
import json
import argparse
import matplotlib.pyplot as plt
%matplotlib inline
%time

import sklearn
from sklearn.model_selection import train_test_split

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Found GPU at: {}'.format(device))

# torch.set_default_tensor_type('torch.cuda.FloatTensor')
# mount gdrive and confrim that dataset exists in this directory
!/opt/bin/nvidia-smi
!ps -aux|grep python

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
root_path = f"/home/hjchris/research/mitigating"
sys.path.append(f"/home/hjchris/repo/pytorch-ssd-master")

from datetime import date
today = !date +"%m%d_%y"
print(f"\nReal Time: {today}")
today = today[0]
print(f"today: {today}")

CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 9.06 µs
Found GPU at: cuda:0
/bin/bash: /opt/bin/nvidia-smi: No such file or directory
root        1027  0.0  0.0  39304 20304 ?        Ss    2021   0:00 /usr/bin/python3 /usr/bin/networkd-dispatcher --run-startup-triggers
root        1166  0.0  0.0 118120 22992 ?        Ssl   2021   0:00 /usr/bin/python3 /usr/share/unattended-upgrades/unattended-upgrade-shutdown --wait-for-signal
hjchris     2788  0.1  0.1 343248 150972 ?       Sl    2021  11:19 /home/hjchris/research/mitigating/.venv/bin/python /home/hjchris/research/mitigating/.venv/bin/jupyter-notebook --no-browser --port=8888
hjchris    16710  0.0  0.0 902444 94368 ?        Ssl   2021   0:31 /home/hjchris/research/mitigating/.venv/bin/python -m ipykernel_launcher -f /home/hjchris/.local/share/jupyter/runtime/kernel-86170f7f-479f-4d41-93c0-1f2fed830e35.json
hjchris   465203 27.2  4.8 24052684 6409100 ?    Ssl  10:15  11:55 /home/hjchris/research/mitigating/.venv/bin/python -m 

In [2]:
file_item=1
logging_folder = f'{root_path}/chris/pascal_ckpts/{today}'
logging_filename = f"{today}-{file_item}.log"

if not os.path.isdir(logging_folder):
    os.mkdir(logging_folder)
    print(f"Created directory: {logging_folder}")

while os.path.exists(os.path.join(logging_folder, logging_filename)):
    file_item += 1
    logging_filename = f"{today}-{file_item}.log"

print(f"File Name: {logging_filename}")
print(f"logging filename: {logging_folder}/{logging_filename}")

# basicConfig = logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s' )
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

streamHandler = logging.StreamHandler(sys.stdout)
streamHandler.setFormatter(formatter)

fileHandler=logging.FileHandler("{0}/{1}".format(logging_folder, logging_filename))
fileHandler.setFormatter(formatter)

train_log = logging.getLogger("train")
train_log.setLevel(logging.INFO)
if train_log.hasHandlers(): 
    train_log.removeHandler(fileHandler)
    train_log.removeHandler(streamHandler)
train_log.addHandler(fileHandler)
train_log.addHandler(streamHandler)

log_dir='/home/hjchris/research/mitigating/chris/pascal_ckpts/evaluations'
inf_log = logging.getLogger("Inference")
inf_log.setLevel(logging.INFO)   

Created directory: /home/hjchris/research/mitigating/chris/pascal_ckpts/0102_22
File Name: 0102_22-1.log
logging filename: /home/hjchris/research/mitigating/chris/pascal_ckpts/0102_22/0102_22-1.log


Customized functions

In [3]:
from vision.utils.misc import str2bool, Timer, freeze_net_layers, store_labels
from vision.ssd.ssd import MatchPrior
from vision.datasets.voc_dataset import VOCDataset
from vision.nn.multibox_loss import MultiboxLoss
from vision.utils import box_utils, measurements

from vision.ssd.config import vgg_ssd_config
from vision.ssd.config import mobilenetv1_ssd_config
from vision.ssd.config import squeezenet_ssd_config
from vision.ssd.data_preprocessing import TrainAugmentation, TestTransform

from vision.ssd.mobilenetv1_ssd import create_mobilenetv1_ssd
from vision.ssd.mobilenetv1_ssd_lite import create_mobilenetv1_ssd_lite
from vision.ssd.squeezenet_ssd_lite import create_squeezenet_ssd_lite
# from vision.ssd.mobilenet_v2_ssd_lite import create_mobilenetv2_ssd_lite

https://discuss.pytorch.org/t/how-to-load-all-data-into-gpu-for-training/27609/5
dataloader GPU

pin_memory

# Data Preprocessing

5011, 4952
Guess: the data originally loaded to DISK. However, for some reasons when they are loaded to RAM the second time 

In [4]:
class Data_Preprocessing:
    def __init__(self, log=print):

        self.widgets = [' [',progressbar.Timer(), '] ', progressbar.Percentage(),
            progressbar.Bar('#'),' (', 
            progressbar.ETA(), ') ', 
            ] 
        self.reset()
        self.log = log
        
    def reset(self):
        self.train_data = []
        self.train_loader = []
        self.valid_data = []
        self.val_loader = []
        
        self.image = 0
        self.boxes = 0
        self.labels = 0

    def add_labels(self):  
        animal_list = [3,8,10,12,13,17] 
        vehicle_list = [1,2,4,6,7,14,19] 
        indoor_list = [5,9,11,16,18,20]
        class_list = animal_list + indoor_list
        class_list = [15]
        print(f"Labels:\nAnimal: {animal_list}\nVehicle:{vehicle_list}\nIndoor: {indoor_list}\n")

        if (valid_data != [] and len(valid_data[0]) == 3) or (train_data != [] and len(train_data[0]) == 3):
            
            if valid_data != []:
                
                labels_dis = []
                label_sum = 0

                for i, data in enumerate(valid_data):
                    image, boxes, labels = data
                    if any(items in labels.unique() for items in class_list):
                        labels_dis.append(1)
                        label_sum += 1
                    else:
                        labels_dis.append(0)

                for i in range(len(labels_dis)):
                    valid_data[i].append(torch.tensor(labels_dis[i]))

                self.log(f"[Valid] Contains any labels {class_list}: {label_sum/len(valid_data)*100:.2f}% is 1.")
                # valid_data = ConcatDataset([valid_data])
            

            if train_data != []:

                labels_dis = []
                label_sum = 0

                for i, data in enumerate(train_data):
                    image, boxes, labels = data
                    if any(items in labels.unique() for items in class_list):
                        labels_dis.append(1)
                        label_sum += 1
                    else:
                        labels_dis.append(0)

                for i in range(len(labels_dis)):
                    train_data[i].append(torch.tensor(labels_dis[i]))

                self.log(f"[Train] Contains any labels {class_list}: {label_sum/len(train_data)*100:.2f}% is 1.")   
                labels_dis = []        
                train_data = ConcatDataset([train_data])

            self.log(f"Added discrimintaor labels: {class_list}.")
            

        self.batch_size = 16
        if train_data != []:
            train_loader = DataLoader(train_data, self.batch_size, shuffle=True, num_workers=4)
            self.log(f"Create train DataLoader from preprocessed data.")
        if valid_data != []:
            val_loader = DataLoader(valid_data, self.batch_size, shuffle=False, num_workers=4)
            self.log(f"Create valid DataLoader from preprocessed data.")
        self.log(f"DataLoader bathsize = {self.batch_size}. Discriminator label {class_list} is already loaded. Loaded data to DataLoaders.")


    def load_online(self, root_path, training = True, validating = True, batch_size = 32, enable_shuffle=True):
        
        self.reset()
        self.batch_size = batch_size
        self.enable_shuffle = enable_shuffle

        #if self.logger: logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        self.log(f"Using Original VOCdataset functions? Training:{training}, Testing:{validating}.")

        if training:
            timer = Timer()
            #  Augmentation/tranformation functions
            dataset_paths = [root_path + '/Pascal_VOC/VOC2012', root_path +'/Pascal_VOC/VOC2007']
            config = mobilenetv1_ssd_config
            train_transform = TrainAugmentation(config.image_size, config.image_mean, config.image_std)
            target_transform = MatchPrior(config.priors, config.center_variance, config.size_variance, 0.5)

            # Prepare Training Data
            self.log(f"Prepare training datasets")
            datasets = []
            datasets_attack = []

            for dp in dataset_paths:
                #  VOCDataset will pass the images into the transformation functions when pulling out the image. Dataset contains [image, boxes, labels]
                dataset = VOCDataset(dp, transform=train_transform, target_transform=target_transform)
                dataset_attack = VOCDataset(dp, transform=train_transform, target_transform=target_transform, is_attack=True)

                label_file = os.path.join(dp, "voc-model-labels.txt")
                store_labels(label_file, dataset.class_names)
                num_classes = len(dataset.class_names)
                datasets.append(dataset)
                datasets_attack.append(dataset_attack)

            # Store labels
            self.log(f"Stored labels into file.")
            self.train_dataset = ConcatDataset(datasets)
            self.train_dataset_attack = ConcatDataset(datasets_attack)
            self.train_loader = DataLoader(self.train_dataset, self.batch_size, shuffle=self.enable_shuffle, num_workers=4)
            self.train_loader_attack = DataLoader(self.train_dataset_attack, self.batch_size, shuffle=self.enable_shuffle, num_workers=4)
            self.log(f"Train data: {len(self.train_dataset)}, loader: {len(self.train_loader)}, batch_size={self.batch_size}, shuffle={self.enable_shuffle}")
            self.log(f"Train attack data : {len(self.train_dataset_attack)}, loader: {len(self.train_loader_attack)}, batch_size={self.batch_size}, shuffle={self.enable_shuffle}")

        if validating:
            #if self.logger: logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
            timer = Timer()

            #  Augmentation/tranformation functions
            dataset_paths = [root_path +'/Pascal_VOC/VOC2007' ,root_path + '/Pascal_VOC/VOC2012']
            validation_dataset_path = root_path + '/Pascal_VOC/VOC2007'
            config = mobilenetv1_ssd_config
            target_transform = MatchPrior(config.priors, config.center_variance, config.size_variance, 0.5)
            test_transform = TestTransform(config.image_size, config.image_mean, config.image_std)
            
            # Prepare Validation Data 
            self.log("Prepare Validation datasets.")
            self.valid_data = VOCDataset(validation_dataset_path, transform=test_transform, target_transform=target_transform, is_test=True)
            self.valid_data_attack = VOCDataset(validation_dataset_path, transform=test_transform, target_transform=target_transform, is_test=True, is_attack=True)
            self.val_loader = DataLoader(self.valid_data, self.batch_size, shuffle=False, num_workers=4)
            self.val_loader_attack = DataLoader(self.valid_data_attack, self.batch_size, shuffle=False, num_workers=4)
            self.log(f"Valid data: {len(self.valid_data)}, loader: {len(self.val_loader)}, batch_size={self.batch_size}, shuffle=False.")
            self.log(f"Valid attack data: {len(self.valid_data_attack)}, loader:{len(self.val_loader_attack)}, batch_size={self.batch_size}, shuffle=False")

# Model


## SSD
one-class classification: https://www.cnblogs.com/fengfenggirl/p/One-Class-Learning.html

1. 14th feature map of mobilnet detects 2166 objects --> feature map size is [576,19,19] --> gird cell = 19x19 (considered small grid) 
2. last feature map of mobilenet detects 600 object --> feature map size is [1280,10,10] --> gird cell = 10x10 (considered small grid) 
3. 1st extra layer detects 150 objects --> feature map size is [512,5,5] --> gird cell = 5x5
4. 2nd extra layer detects 54 objects --> feature map size is [256,3,3] --> gird cell = 3x3
5. 3rd extra layer detects 24 objects --> feature map size is [256,2,2] --> gird cell = 2x2
6. 4th extra layer detects 6 objects --> feature map size is [64,1,1] --> gird cell = 1x1
 


total: 2166+600+150+54+24+6 = 3000

In [5]:
def dec_wrapper(norm):
    def hook_func(grad):
        norm.append(math.pow(grad.norm().data[0], 2))
        pass
    return hook_func

def adv_wrapper(norm = [], grad_scale = 2):
    def hook_func(grad):
        new_grad = -grad * gamma
        norm.append(math.pow(new_grad.norm().data, 2))
        return new_grad
        pass
    return hook_func

def adv_hook_func(grad):
    new_grad = -grad * 2
    return new_grad

In [6]:
from vision.utils import box_utils
from collections import namedtuple

GraphPath = namedtuple("GraphPath", ['s0', 'name', 's1'])  
class SSD(nn.Module):
    def __init__(self, num_classes: int, base_net: nn.ModuleList, source_layer_indexes: List[int], extras: nn.ModuleList, 
                 classification_headers: nn.ModuleList, regression_headers: nn.ModuleList,
                 encoder_headers: nn.ModuleList, is_test=False, config=None
                 ):
        """Compose a SSD model using the given components.
        feature_map = -3: discriminator connecting to raw images X.
        feature_map = -1: adding extra layers as encoder.
        """
        super(SSD, self).__init__()

        # SSD parameters
        self.num_classes = num_classes
        self.base_net = base_net
        self.source_layer_indexes = source_layer_indexes
        self.extras = extras
        self.classification_headers = classification_headers
        self.regression_headers = regression_headers
        self.config = config

        # outputs
        self.output_reset()

        # register layers in source_layer_indexes by adding them to a module list
        self.source_layer_add_ons = nn.ModuleList([t[1] for t in source_layer_indexes if isinstance(t, tuple) and not isinstance(t, GraphPath)])

        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        
        self.is_inference = is_test
        
        if self.is_inference:
            self.config = config
            self.priors = config.priors.to(self.device)
            self.eval()

        # Encoder 
        self.encoder_headers = encoder_headers

    def output_reset(self):  
        
        self.confidences = []
        self.locations = []

    def forward(self, x: torch.Tensor, discriminator_loc) -> Tuple[torch.Tensor, torch.Tensor]:
        
        self.output_reset()
        discriminator_loc = discriminator_loc
        start_layer_index = 0
        header_index = 0
        dis_index = 0

        # ======================================================================
        # [Base net] Mobilenetv2
        # ======================================================================
        for end_layer_index in self.source_layer_indexes:

            #  Continue to a feature layer INDEX (end_layer_index)
            if isinstance(end_layer_index, GraphPath):
                path = end_layer_index
                end_layer_index = end_layer_index.s0
                added_layer = None
            elif isinstance(end_layer_index, tuple):
                added_layer = end_layer_index[1]
                end_layer_index = end_layer_index[0]
                path = None
            else:
                added_layer = None
                path = None

            #  Go to that feature layer of the index (from basenet --> mobilenet)
            for layer in self.base_net[start_layer_index: end_layer_index]:
                if (dis_index < 18 and dis_index == discriminator_loc):
                    if dis_index == 0:
                        x = layer(x)
                        z = x
                    else:
                        sub = getattr(self.base_net[dis_index], 'conv')
                        for sublayer in sub[:3]:
                            x = sublayer(x)
                        z = x
                        for sublayer in sub[3:]:
                            x = sublayer(x)         
                else:
                    x = layer(x)

                if dis_index < 18:
                    dis_index += 1
                
            # if there is any added layers
            y = added_layer(x) if added_layer else x

            #  Block 14
            if path:
                sub = getattr(self.base_net[end_layer_index], path.name)
                for layer in sub[:path.s1]:
                    x = layer(x)
                y = x
                for layer in sub[path.s1:]:
                    x = layer(x)
                end_layer_index += 1
            
            if dis_index == discriminator_loc: z = y
                
            #  Block 14 and Block 18 to Predictor
            start_layer_index = end_layer_index
            confidence, location = self.compute_header(header_index, y)
            self.confidences.append(confidence)
            self.locations.append(location)

            #  Discriminator for the current feature (now only for the first feature)  
            dis_index += 1
            header_index += 1
        
        # ======================================================================
        #  Go through the rest of the basnet if any--> mobilenet 
        # ======================================================================
        for layer in self.base_net[end_layer_index:]:
            x = layer(x)

        # ======================================================================
        # Extra layers
        # ======================================================================
        # Use extra layer features to compute the confidence and location
        for layer in self.extras:
            x = layer(x)
            confidence, location = self.compute_header(header_index, x)
            self.confidences.append(confidence)
            self.locations.append(location)
            
            if dis_index == discriminator_loc: z = x

            #  Discriminator for the extra features (now only for the first feature)
            dis_index += 1
            header_index += 1
            
        # ======================================================================
        #  Concatenate the results
        # ======================================================================
        self.confidences = torch.cat(self.confidences, 1)
        self.locations = torch.cat(self.locations, 1)
            
        # ======================================================================
        # Return the results
        # ======================================================================
        if self.is_inference:
            self.confidences = F.softmax(self.confidences, dim=2)
            self.boxes = box_utils.convert_locations_to_boxes(
                self.locations, self.priors, self.config.center_variance, self.config.size_variance
            )
            self.boxes = box_utils.center_form_to_corner_form(self.boxes)
            return self.confidences, self.boxes, z
        else:
            return self.confidences, self.locations, z

    ####################################################################################################

    # computer classification and regression layers
    def compute_header(self, i, x):
        #  Classification for confidence (6 sets of 21 numbers for each grid)
        confidence = self.classification_headers[i](x)
        confidence = confidence.permute(0, 2, 3, 1).contiguous()
        confidence = confidence.view(confidence.size(0), -1, self.num_classes)
        
        #  Regression for location (6 sets of 4 numbers for each grid)
        location = self.regression_headers[i](x)
        location = location.permute(0, 2, 3, 1).contiguous()
        location = location.view(location.size(0), -1, 4)

        return confidence, location

    def init_base_net(self, model):
        self.base_net.load_state_dict(torch.load(model, map_location=lambda storage, loc: storage), strict=True)
        self.source_layer_add_ons.apply(_xavier_init_)
        self.extras.apply(_xavier_init_)
        self.classification_headers.apply(_xavier_init_)
        self.regression_headers.apply(_xavier_init_)
        if self.encoder_headers: self.encoder_headers.apply(_xavier_init_)

    def init_from_pretrained_ssd(self, model_path, log=print):
        state_dict = torch.load(model_path, map_location=lambda storage, loc: storage)
        state_dict = state_dict['model_state_dict']
        # state_dict = {k: v for k, v in state_dict.items() if not (k.startswith("classification_headers") or k.startswith("regression_headers"))}
        state_dict = {k: v for k, v in state_dict.items() if not (k.startswith("discriminator_headers") or k.startswith("encoder_headers"))}
        model_dict = self.state_dict()
        model_dict.update(state_dict)
        self.load_state_dict(model_dict)
        log(f"Loaded pretrained SSD.")
            
    def init_encoder(self, model):
        if self.encoder_headers:
            state_dict = torch.load(model, map_location=lambda storage, loc: storage)
            state_dict = state_dict['model_state_dict']
            state_dict = {k: v for k, v in state_dict.items() if (k.startswith("encoder_headers"))}
            model_dict = self.state_dict()
            model_dict.update(state_dict)
            self.load_state_dict(model_dict)
            train_log.info(f"Loaded pretrained encoder from {model}.")

    def init(self):
        self.base_net.apply(_xavier_init_)
        self.source_layer_add_ons.apply(_xavier_init_)
        self.extras.apply(_xavier_init_)
        self.classification_headers.apply(_xavier_init_)
        self.regression_headers.apply(_xavier_init_)
        if self.encoder_headers: self.encoder_headers.apply(_xavier_init_)

    def load(self, model):
        self.load_state_dict(torch.load(model, map_location=lambda storage, loc: storage))

    def save(self, privacy_flag, public_flag, alpha, max_grad_norm, epoch, optimizers, schedulers, val_loss, val_d_ent, val_d_loss, 
             sensitive_classes, model_path, discriminator_loc=None, discriminator=None, public=None, val_p_loss=None, public_classes=None,
             arl_setting = None):
        dic = {'privacy_flag': privacy_flag, 
               'public_flag': public_flag,
               'epoch': epoch,
               'model_state_dict': self.state_dict(),
               'max_grad_norm': max_grad_norm, 
               'optimizer_state_dict': optimizers[0].optimizer.state_dict(),
               'scheduler_stat_dict': schedulers[0].state_dict(),
               'loss': val_loss,
              }
        
        if discriminator:
            dic['alpha'] = alpha
            dic['arl_setting'] = arl_setting
            dic['sensitive_classes'] = sensitive_classes
            dic['discriminator_loc'] = discriminator_loc
            dic['entropy'] = val_d_ent
            dic['discrimiantor_loss'] = val_d_loss
            dic['discriminator_optimizer_state_dict'] = optimizers[1].optimizer.state_dict()
            dic['discriminator_scheduler_state_dict'] = schedulers[1].state_dict()
            dic['discriminator_state_dict'] = discriminator.state_dict()
            
        if public_flag:
            dic['public_state_dict'] = public.state_dict()
            dic['public_optimizer_state_dict'] = optimizers[2].optimizer.state_dict()
            dic['public_scheduler_state_dict'] = schedulers[2].state_dict()
            dic['public_loss'] = val_p_loss
            dic['public_classes'] = public_classes
            
        torch.save(dic, model_path)
        
    def save_attack(self, privacy_flag, alpha, max_grad_norm, epoch, optimizers, schedulers, ssd_loss, attacker_loss, 
                    sensitive_classes, entropy, model_path, discriminator_loc=None, discriminator=None,
                    attacker_loc=None, attacker=None, name=None):
        
        dic = {'privacy_flag': True, 
               'epoch': epoch,
               'model_state_dict': self.state_dict(),
               'max_grad_norm': max_grad_norm, 
               'loss': ssd_loss,
               'public_flag': False
              }

        
        dic['attacked_model'] = name
        dic['alpha'] = alpha
        dic['sensitive_classes'] = sensitive_classes
        dic['attacker_loss'] = attacker_loss
        dic['attacker_loc'] = attacker_loc
        dic['attacker_optimizer_state_dict'] = optimizers[1].optimizer.state_dict()
        dic['attacker_scheduler_state_dict'] = schedulers[1].optimizer.state_dict()
        dic['attacker_state_dict'] = attacker.state_dict()
        dic['entropy'] = entropy
            
        torch.save(dic, model_path)
        
class MatchPrior(object):
    
    def __init__(self, center_form_priors, center_variance, size_variance, iou_threshold):
        self.center_form_priors = center_form_priors
        self.corner_form_priors = box_utils.center_form_to_corner_form(center_form_priors)
        self.center_variance = center_variance
        self.size_variance = size_variance
        self.iou_threshold = iou_threshold

    def __call__(self, gt_boxes, gt_labels):
        if type(gt_boxes) is np.ndarray:
            gt_boxes = torch.from_numpy(gt_boxes)
        if type(gt_labels) is np.ndarray:
            gt_labels = torch.from_numpy(gt_labels)
        boxes, labels = box_utils.assign_priors(gt_boxes, gt_labels,
                                                self.corner_form_priors, self.iou_threshold)
        boxes = box_utils.corner_form_to_center_form(boxes)
        locations = box_utils.convert_boxes_to_locations(boxes, self.center_form_priors, self.center_variance, self.size_variance)
        return locations, labels

def _xavier_init_(m: nn.Module):
    if isinstance(m, nn.Conv2d):
        nn.init.xavier_uniform_(m.weight)

def freeze_net_layers(net):
    for param in net.parameters():
        param.requires_grad = False

BatchNorm might still cause the gradient updates:https://discuss.pytorch.org/t/why-is-it-when-i-call-require-grad-false-on-all-my-params-my-weights-in-the-network-would-still-update/22126/16

vgg
borrowed from https://github.com/amdegroot/ssd.pytorch/blob/master/ssd.py

## mobilenetv2

In [7]:
# Modified from https://github.com/tonylins/pytorch-mobilenet-v2/blob/master/MobileNetV2.py.
# In this version, Relu6 is replaced with Relu to make it ONNX compatible.
# BatchNorm Layer is optional to make it easy do batch norm confusion.


def conv_bn(inp, oup, stride, use_batch_norm=True, onnx_compatible=False):
    ReLU = nn.ReLU if onnx_compatible else nn.ReLU6

    if use_batch_norm:
        return nn.Sequential(
            nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
            nn.BatchNorm2d(oup),
            ReLU(inplace=True)
        )
    else:
        return nn.Sequential(
            nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
            ReLU(inplace=True)
        )


def conv_1x1_bn(inp, oup, use_batch_norm=True, onnx_compatible=False):
    ReLU = nn.ReLU if onnx_compatible else nn.ReLU6
    if use_batch_norm:
        return nn.Sequential(
            nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
            nn.BatchNorm2d(oup),
            ReLU(inplace=True)
        )
    else:
        return nn.Sequential(
            nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
            ReLU(inplace=True)
        )


class InvertedResidual(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio, use_batch_norm=True, onnx_compatible=False):
        super(InvertedResidual, self).__init__()
        ReLU = nn.ReLU if onnx_compatible else nn.ReLU6

        self.stride = stride
        assert stride in [1, 2]

        hidden_dim = round(inp * expand_ratio)
        self.use_res_connect = self.stride == 1 and inp == oup

        if expand_ratio == 1:
            if use_batch_norm:
                self.conv = nn.Sequential(
                    # dw
                    nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                    nn.BatchNorm2d(hidden_dim),
                    ReLU(inplace=True),
                    # pw-linear
                    nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                    nn.BatchNorm2d(oup),
                )
            else:
                self.conv = nn.Sequential(
                    # dw
                    nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                    ReLU(inplace=True),
                    # pw-linear
                    nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                )
        else:
            if use_batch_norm:
                self.conv = nn.Sequential(
                    # pw (pointwise)
                    nn.Conv2d(inp, hidden_dim, 1, 1, 0, bias=False),
                    nn.BatchNorm2d(hidden_dim),
                    ReLU(inplace=True),
                    # dw (depthwise)
                    nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                    nn.BatchNorm2d(hidden_dim),
                    ReLU(inplace=True),
                    # pw-linear
                    nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                    nn.BatchNorm2d(oup),
                )
            else:
                self.conv = nn.Sequential(
                    # pw (pointwise)
                    nn.Conv2d(inp, hidden_dim, 1, 1, 0, bias=False),
                    ReLU(inplace=True),
                    # dw (depthwise)
                    nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                    ReLU(inplace=True),
                    # pw-linear
                    nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                )

    def forward(self, x):
        if self.use_res_connect:
            return x + self.conv(x)
        else:
            return self.conv(x)


class MobileNetV2(nn.Module):
    def __init__(self, n_class=1000, input_size=224, width_mult=1., dropout_ratio=0.2,
                 use_batch_norm=True, onnx_compatible=False):
        super(MobileNetV2, self).__init__()
        block = InvertedResidual
        input_channel = 32
        last_channel = 1280
        interverted_residual_setting = [
            # t, c, n, s
            [1, 16, 1, 1],
            [6, 24, 2, 2],
            [6, 32, 3, 2],
            [6, 64, 4, 2],
            [6, 96, 3, 1],
            [6, 160, 3, 2],
            [6, 320, 1, 1],
        ]

        # building first layer
        assert input_size % 32 == 0
        input_channel = int(input_channel * width_mult)
        self.last_channel = int(last_channel * width_mult) if width_mult > 1.0 else last_channel
        self.features = [conv_bn(3, input_channel, 2, onnx_compatible=onnx_compatible)]
        # building inverted residual blocks
        for t, c, n, s in interverted_residual_setting:
            output_channel = int(c * width_mult)
            for i in range(n):
                if i == 0:
                    self.features.append(block(input_channel, output_channel, s,
                                               expand_ratio=t, use_batch_norm=use_batch_norm,
                                               onnx_compatible=onnx_compatible))
                else:
                    self.features.append(block(input_channel, output_channel, 1,
                                               expand_ratio=t, use_batch_norm=use_batch_norm,
                                               onnx_compatible=onnx_compatible))
                input_channel = output_channel
        # building last several layers
        self.features.append(conv_1x1_bn(input_channel, self.last_channel,
                                         use_batch_norm=use_batch_norm, onnx_compatible=onnx_compatible))
        # make it nn.Sequential
        self.features = nn.Sequential(*self.features)

        # building classifier
        self.classifier = nn.Sequential(
            nn.Dropout(dropout_ratio),
            nn.Linear(self.last_channel, n_class),
        )

        self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = x.mean(3).mean(2)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                n = m.weight.size(1)
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()

def SeperableConv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, onnx_compatible=False):
    """Replace Conv2d with a depthwise Conv2d and Pointwise Conv2d.
    """
    ReLU = nn.ReLU if onnx_compatible else nn.ReLU6
    return Sequential(
        Conv2d(in_channels=in_channels, out_channels=in_channels, kernel_size=kernel_size,
               groups=in_channels, stride=stride, padding=padding),
        BatchNorm2d(in_channels),
        ReLU(),
        Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=1),
    )

## ssd_create

* default: 6 boxes

* MobileNetV2.features = [conv_bn(3, input_channel, 2, onnx_compatible=onnx_compatible)]

* extras --> used for the last layer feature map


In [8]:
from torch.nn import Conv2d, Sequential, ModuleList, BatchNorm2d

class create_mobilenetv2_ssd_lite:

    def __init__(self, num_classes, width_mult=1.0, use_batch_norm=True, onnx_compatible=False, is_test=False):

        self.num_classes = num_classes
        self.is_test = is_test

        # networks
        self.base_net = MobileNetV2(width_mult=width_mult, use_batch_norm=use_batch_norm, onnx_compatible=onnx_compatible).features
        self.source_layer_indexes = [GraphPath(14, 'conv', 3),19,]
        self.extras = ModuleList([
            InvertedResidual(1280, 512, stride=2, expand_ratio=0.2),
            InvertedResidual(512, 256, stride=2, expand_ratio=0.25),
            InvertedResidual(256, 256, stride=2, expand_ratio=0.5),
            InvertedResidual(256, 64, stride=2, expand_ratio=0.25)
            ])
        
        self.regression_headers = ModuleList([
            SeperableConv2d(in_channels=round(576 * width_mult), out_channels=6 * 4, kernel_size=3, padding=1, onnx_compatible=False),
            SeperableConv2d(in_channels=1280, out_channels=6 * 4, kernel_size=3, padding=1, onnx_compatible=False),
            SeperableConv2d(in_channels=512, out_channels=6 * 4, kernel_size=3, padding=1, onnx_compatible=False),
            SeperableConv2d(in_channels=256, out_channels=6 * 4, kernel_size=3, padding=1, onnx_compatible=False),
            SeperableConv2d(in_channels=256, out_channels=6 * 4, kernel_size=3, padding=1, onnx_compatible=False),
            Conv2d(in_channels=64, out_channels=6 * 4, kernel_size=1),
            ])

        self.classification_headers = ModuleList([
            SeperableConv2d(in_channels=round(576 * width_mult), out_channels=6 * num_classes, kernel_size=3, padding=1),
            SeperableConv2d(in_channels=1280, out_channels=6 * num_classes, kernel_size=3, padding=1),
            SeperableConv2d(in_channels=512, out_channels=6 * num_classes, kernel_size=3, padding=1),
            SeperableConv2d(in_channels=256, out_channels=6 * num_classes, kernel_size=3, padding=1),
            SeperableConv2d(in_channels=256, out_channels=6 * num_classes, kernel_size=3, padding=1),
            Conv2d(in_channels=64, out_channels=6 * num_classes, kernel_size=1),
            ])

        self.encoder_headers = ModuleList()
        
    def create_SSD(self):
        
        return SSD(self.num_classes,
                   self.base_net, 
                   self.source_layer_indexes, 
                   self.extras, 
                   self.classification_headers, 
                   self.regression_headers, 
                   self.encoder_headers,
                   is_test= self.is_test, 
                   config = mobilenetv1_ssd_config
                   )

    def create_SSD1(self):
        
        return 0

def create_mobilenetv2_ssd_lite_predictor(net, candidate_size=200, nms_method=None, sigma=0.5, device=torch.device('cpu'), config = mobilenetv1_ssd_config):
    predictor = Predictor(net, config.image_size, config.image_mean,
                          config.image_std,
                          nms_method=nms_method,
                          iou_threshold=config.iou_threshold,
                          candidate_size=candidate_size,
                          sigma=sigma,
                          device=device)
    return predictor

# Discriminator

In [9]:
class Flatten(torch.nn.Module):
    def forward(self, x):
        batch_size = x.shape[0]
        return x.view(batch_size, -1)

In [10]:
class Create_Discriminator():
    def __init__(self):
        self.discsriminator_headers = ModuleList()
        self.discriminator_A_settings = {
                                         0:[32, 22500],   2:[96, 1000],
                                         7:[192, 250],  11:[384, 90],  14:[576, 90],
                                         18:[1280, 40], 19:[512, 10],   20:[256, 10],
                                         21:[256, 10],    22:[64, 40]
                                         }
        
    def build(self, d_loc):
        input_size = self.discriminator_A_settings[d_loc]
        return Discriminator(cnn_input=input_size[0], linear_input=input_size[1])
    
class Discriminator(nn.Module):
    def __init__(self, cnn_input, linear_input, d_type="A"):
        super(Discriminator, self).__init__()
        self.net_headers = self.A(cnn_input, linear_input)
        self.net_headers.apply(_xavier_init_)
        self.use_cuda = torch.cuda.is_available()
        
        if self.use_cuda:
            self.net_headers = self.net_headers.cuda()
        
    def forward(self, x):
        for layer in self.net_headers:
            x = layer(x)
        return x
    
#     def load(self, path, is_attacker=False):
#         state_dict = torch.load(path, map_location=lambda storage, loc: storage)
#         state_dict = state_dict['discriminator_state_dict'] if not is_attacker else state_dict['attacker_state_dict']
#         # state_dict = {k: v for k, v in state_dict.items()} #(k.startswith("encoder_headers")
#         #print(len(state_dict.keys()))
#         model_dict = self.state_dict()
#         model_dict.update(state_dict)
#         self.load_state_dict(model_dict)
        
    def load(self, path, module="discriminator"):
        state_dict = torch.load(path, map_location=lambda storage, loc: storage)
        
        if module == "discriminator":
            state_dict = state_dict['discriminator_state_dict'] 
        elif module == "attacker":
            state_dict = state_dict['attacker_state_dict']
        elif module == "public": 
            state_dict = state_dict['public_state_dict']
        else:
            raise Exception(f"Only discriminator, attacker, or public.")
        # state_dict = {k: v for k, v in state_dict.items()} #(k.startswith("encoder_headers")
        #print(len(state_dict.keys()))
        model_dict = self.state_dict()
        model_dict.update(state_dict)
        self.load_state_dict(model_dict)
    
    
    def load_discriminator(self, model):
        if self.discriminator_headers:
            self.discriminator_headers.apply(_xavier_init_)
            if self.feature_map == -1 or self.feature_map == 24:
                self.discriminator_headers[0].load_state_dict(torch.load(model, map_location=lambda storage, loc: storage), strict=True)
                print(f"Loaded {discriminator_type} in discriminator.") # load image_net 71.8
        else:
            print("No discriminator to load.")
            
    def init_attacker(self, net):
        if self.discriminator_headers:
            state_dict = torch.load(model, map_location=lambda storage, loc: storage)
            state_dict = state_dict['model_state_dict']
            state_dict = {k: v for k, v in state_dict.items() if (k.startswith("discriminator_headers"))}
            model_dict = self.state_dict()
            model_dict.update(state_dict)
            self.load_state_dict(model_dict)
            print(f"Loaded attacker from {model[20:]}.") # load image_net 71.8
        else:
            print("No attacker to load.")
        
    def A(self, cnn_input=1280, linear_input=10, width_mult=1.0, use_batch_norm=True, onnx_compatible=False):
        return ModuleList([
                    InvertedResidual(cnn_input, 512, stride=2, expand_ratio=0.2),
                    InvertedResidual(512, 256, stride=2, expand_ratio=0.25),
#                     InvertedResidual(256, 256, stride=2, expand_ratio=0.5),
                    InvertedResidual(256, 64, stride=2, expand_ratio=0.25),
                    SeperableConv2d(in_channels=64, out_channels=32, kernel_size=3, padding=1),
                    SeperableConv2d(in_channels=32, out_channels=10, kernel_size=3, padding=1),
                    Flatten(),
                    nn.Linear(linear_input, 2)
                ])
    
    def BE(self, encoder_expand_ratio=12):           
        return ModuleList([
                    SeperableConv2d(in_channels=3*encoder_expand_ratio, out_channels=10, kernel_size=5, padding=0, stride=3),
                    nn.Linear(98010,2)
                ])

    def B(self):

        #             discriminator_headers = ModuleList([
        #                 SeperableConv2d(in_channels=3*encoder_expand_ratio, out_channels=10, kernel_size=5, padding=0, stride=3),
        #                 nn.Linear(98010,2)
        #             ])

        #             discriminator_headers = ModuleList([
        #                 InvertedResidual(3, 8, stride=2, expand_ratio=6),
        #                 InvertedResidual(8, 16, stride=2, expand_ratio=6),
        #                 InvertedResidual(16, 32, stride=2, expand_ratio=4),
        #                 InvertedResidual(32, 64, stride=2, expand_ratio=3),
        #                 InvertedResidual(64, 64, stride=2, expand_ratio=3),
        #                 SeperableConv2d(in_channels=64, out_channels=128, kernel_size=3, padding=0, stride=2),
        #                 BatchNorm2d(128),
        #                 nn.ReLU(inplace=True),
        #                 SeperableConv2d(in_channels=128, out_channels=256, kernel_size=3, padding=0, stride=2),
        #                 nn.Linear(256,2)
        #             ])

        return ModuleList([
                    SeperableConv2d(in_channels=3, out_channels=10, kernel_size=5, padding=0, stride=3),
                    nn.Linear(98010,2)
                    ])

    def vgg(self):
        vgg_base_net = ModuleList(vgg(vgg_config))
        vgg_base_net = nn.Sequential(*vgg_base_net)
        return ModuleList([
                    vgg_base_net,
                    BatchNorm2d(1024),
                    SeperableConv2d(in_channels=1024, out_channels=512, kernel_size=5, padding=0, stride=3),
                    nn.Linear(12800,2)
                    ])

    def ssd(self, width_mult=1.0, use_batch_norm=True, onnx_compatible=False):
        base_net_dis = MobileNetV2(width_mult=width_mult, use_batch_norm=use_batch_norm, onnx_compatible=onnx_compatible).features
        return ModuleList([
                            base_net_dis,
                            InvertedResidual(1280, 512, stride=2, expand_ratio=0.2),
                            InvertedResidual(512, 256, stride=2, expand_ratio=0.25),
                            InvertedResidual(256, 256, stride=2, expand_ratio=0.5),
                            InvertedResidual(256, 64, stride=2, expand_ratio=0.25),
                            SeperableConv2d(in_channels=64, out_channels=10, kernel_size=3, padding=1),
                            nn.Linear(10, 2)
                            ]) 

# Losses

Classification: hard negative mining --> if an image has y classes, choose 3y backgrounds based on the loss magnitude

Regression: set the background boxes to zero. (because we don't need to update the box size of the background)

## MultiboxLoss

In [11]:
from vision.utils import box_utils

class MultiboxLoss(nn.Module):
    def __init__(self, priors, iou_threshold, neg_pos_ratio,
                 center_variance, size_variance, device):
        """Implement SSD Multibox Loss.
        Basically, Multibox loss combines classification loss
         and Smooth L1 regression loss.
        """
        super(MultiboxLoss, self).__init__()
        self.iou_threshold = iou_threshold
        self.neg_pos_ratio = neg_pos_ratio
        self.center_variance = center_variance
        self.size_variance = size_variance
        # self.priors = priors
        # self.priors.to(device)


        self.use_cuda = torch.cuda.is_available()
        self.regression_loss=torch.FloatTensor(0)
        self.classification_loss=torch.FloatTensor(0)
        self.grad_disc_out = torch.zeros(0)

        if self.use_cuda:
            self.regression_loss=self.regression_loss.cuda()
            self.classification_loss=self.classification_loss.cuda()
            self.grad_disc_out=self.grad_disc_out.cuda()

    def forward(self, confidence, predicted_locations, labels, gt_locations, 
                dis_prediction = 0, dis_gt_labels = 0, original_flag = True, eval = False, freeze_ssd = False):

        """Compute classification loss and smooth l1 loss.
        Args:
            confidence (batch_size, num_priors, num_classes): class predictions.
            locations (batch_size, num_priors, 4): predicted locations.
            labels (batch_size, num_priors): real labels of all the priors.
            boxes (batch_size, num_priors, 4): real boxes corresponding all the priors.
        """

        num_classes = confidence.size(2)

        with torch.no_grad():
            # derived from cross_entropy=sum(log(p))
            loss = -F.log_softmax(confidence, dim=2)[:, :, 0]
            """ Why does it only choose the first one? """
            mask = hard_negative_mining(loss, labels, self.neg_pos_ratio)
        
        #  Total Classification Loss
        confidence = confidence[mask, :]
        if not original_flag: confidence = Variable(confidence.data, requires_grad=(not eval and not freeze_ssd)).contiguous()
        self.classification_loss = F.cross_entropy(confidence.reshape(-1, num_classes), labels[mask], reduction='sum')

        #  Total Regression Loss 
        pos_mask = labels > 0
        predicted_locations = predicted_locations[pos_mask, :].reshape(-1, 4)
        if not original_flag: predicted_locations = Variable(predicted_locations.data, requires_grad=(not eval and not freeze_ssd)).contiguous()
        gt_locations = gt_locations[pos_mask, :].reshape(-1, 4)
        smooth_l1_loss = F.smooth_l1_loss(predicted_locations, gt_locations, reduction='sum')

        # Average Losses
        num_pos = gt_locations.size(0)
        
        
        if not original_flag:
            self.classification_loss = torch.true_divide(self.classification_loss, num_pos)
            self.regression_loss = torch.true_divide(smooth_l1_loss, num_pos)
            return self.classification_loss, self.regression_loss
        
        else:
            return self.classification_loss/num_pos, smooth_l1_loss/num_pos


In [12]:
def hard_negative_mining(loss, labels, neg_pos_ratio):
    """
    It used to suppress the presence of a large number of negative prediction.
    It works on image level not batch level.
    For any example/image, it keeps all the positive predictions and
     cut the number of negative predictions to make sure the ratio
     between the negative examples and positive examples is no more
     the given ratio for an image.
    Args:
        loss (N, num_priors): the loss for each example.
        labels (N, num_priors): the labels.
        neg_pos_ratio:  the ratio between the negative examples and positive examples.
    """
    pos_mask = labels > 0
    num_pos = pos_mask.long().sum(dim=1, keepdim=True) # the number of positive label in an image
    num_neg = num_pos * neg_pos_ratio # the total number of labels to use in an image

    loss[pos_mask] = -math.inf
    _, indexes = loss.sort(dim=1, descending=True)
    _, orders = indexes.sort(dim=1)
    neg_mask = orders < num_neg
    return pos_mask | neg_mask

## Entropy

In [13]:
import torch
from  torch.nn.modules.loss import _Loss

class EntropyLoss(_Loss):
    def __init__(self, size_average=None, reduce=None, reduction='mean'):
        super(EntropyLoss, self).__init__(size_average, reduce, reduction)

    # input is probability distribution of output classes
    def forward(self, input):
        if (input < 0).any() or (input > 1).any():
            raise Exception('Entropy Loss takes probabilities 0<=input<=1')

        input = input + 1e-16  # for numerical stability while taking log
        H = torch.mean(torch.sum(input * torch.log(input), dim=1))
        
        return H
    
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.reshape(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

# Optimizer

param = list(fc1.parameters()) + list(fc2.parameters())

In [14]:
class Optim(object):

    def _makeOptimizer(self):
        if self.method == 'sgd':
            self.optimizer = optim.SGD(self.params, lr=self.lr, momentum=self.sgd_momentum, weight_decay=self.sgd_weight_decay)
        elif self.method == 'adagrad':
            self.optimizer = optim.Adagrad(self.params, lr=self.lr)
        elif self.method == 'adadelta':
            self.optimizer = optim.Adadelta(self.params, lr=self.lr)
        elif self.method == 'adam':
            self.optimizer = optim.Adam(self.params, lr=self.lr, betas=(self.adam_momentum, 0.999))
        else:
            raise RuntimeError("Invalid optim method: " + self.method)

    def __init__(self, params, method, enc_params = [0],  max_grad_norm = 5, lr = 0.001, lr_decay=1,
                 start_decay_at= 60, adam_momentum=0.9, sgd_momentum = 0.9, sgd_weight_decay = 5e-4, privacy_flag=False):
        
        self.params = params  # careful: params may be a generator
        self.encoder_params = list(enc_params)
        self.last_ppl = None
        self.lr = lr
        self.max_grad_norm = max_grad_norm
        self.method = method
        self.lr_decay = lr_decay
        self.start_decay_at = start_decay_at
        self.start_decay = False
        self.adam_momentum = adam_momentum
        self.sgd_momentum = sgd_momentum
        self.sgd_weight_decay = sgd_weight_decay
        self._makeOptimizer()

    def step_all(self):
        #  Calculate encoder gradient norm
        
        enc_grad_norm = 0.0
        for param in self.encoder_params:
            enc_grad_norm += math.pow(param.grad.data.norm(), 2) if param.grad != None else 0.0
        enc_grad_norm = math.sqrt(enc_grad_norm)

        #  Calculate all gradient norm
        grad_norm = 0.0
        for param in self.params:
            for p in param['params']:
                grad_norm += math.pow(p.grad.data.norm(), 2) 
        grad_norm = math.sqrt(grad_norm)

        #  Shrink the graident if the gradients are too large
        shrinkage = self.max_grad_norm / grad_norm
        for param in self.params:
            for p in param['params']:
                if shrinkage < 1:
                    p.grad.data.mul_(shrinkage)

        #  Start updating
        self.optimizer.step()
        
        return enc_grad_norm, grad_norm, shrinkage

# Main

## maxent_network


https://www.zhihu.com/question/61044004 register_hook

In [15]:
class AverageMeter(object):
    """Computes and stores the average and current value
       Imported from https://github.com/pytorch/examples/blob/master/imagenet/main.py#L247-L262
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0.0
        self.avg = 0.0
        self.sum = 0.0
        self.count = 0.0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [16]:
class Maxtrain_Network:
    def __init__(self, nets, optimizers, train_loader, test_loader, alpha = None, sensitive_class = None, public_classes = None, log=print):
        
        self.log=log
        self.net = nets[0]                
        self.discriminator = nets[1]
        self.public = nets[2]
        
        self.optimizer = optimizers[0]
        self.discriminator_optimizer = optimizers[1]
        self.public_optimizer = optimizers[2]
        
        self.alpha = alpha
        self.sensitive_classes = sensitive_class
        self.public_classes = public_classes
        
        self.entropy_loss = EntropyLoss()
        self.softmaxlayer = nn.Softmax(dim=1)
        self.logsoftmax = nn.LogSoftmax(dim=1)
        self.nll_loss = nn.NLLLoss()
        self.mse_loss = nn.MSELoss()
        
        self.train_loader = train_loader
        self.test_loader = test_loader

        self.use_cuda = torch.cuda.is_available()
        self.images = torch.zeros(0, 0, 0)
        self.boxes = torch.zeros(0, 0, 0)
        self.labels = torch.zeros(0, dtype=torch.long)
        self.sensitive_labels = torch.zeros(0, dtype=torch.long)
        self.public_labels = torch.zeros(0, dtype=torch.long)
        self.grad_norm = torch.tensor(1.0)


        if self.use_cuda:
            self.images=self.images.cuda()
            self.boxes=self.boxes.cuda()
            self.labels=self.labels.cuda()
            self.grad_norm=self.grad_norm.cuda()

            if self.discriminator:
                self.sensitive_labels = self.sensitive_labels.cuda()
                self.alpha = self.alpha.to(DEVICE) 
            
            if self.public:
                self.public_labels = self.public_labels.cuda()
        
        self.meters_reset()

    def meters_reset(self):
        self.running_loss = 0.0
        self.running_regression_loss = 0.0
        self.running_classification_loss = 0.0
        self.running_regression_loss_dis = 0.0
        self.running_discriminator_loss = 0.0
        self.report_grad_norm = 0.0
        self.running_ssd_loss = 0.0
        self.running_discriminator_entropy = 0.0
        self.running_public_loss = 0.0

    def online_find_labels(self, labels, target_classes=[15]):
        label_temp = torch.zeros(len(labels), dtype=torch.long)
        for j in range(len(label_temp)):
            label_temp[j] = (1 if any(items in labels[j].unique() for items in target_classes) else 0)
        return label_temp

    def compute_gradnorm(self):
        grad_sum = 0
        for param in list(filter(lambda p: p.grad is not None, self.net.parameters())):
            grad_sum += math.pow(param.grad.data.norm(), 2) 
        return math.sqrt(grad_sum)
    
    def call_param(self, net_header, name, print_out=False):
        all_param = [param.detach().clone() for param in net_header.parameters()] 
        if print_out: print(f"{name}:{all_param[0][0][0]}")
        return all_param

    def assert_param(self, p1, p2):
        assert len(p1) == len(p2)
        for i in range(len(p1)): assert torch.sum(p1[i] == p2[i]) == p1[i].numel()

    def grad_summation(self, grads):
        grad_sum = 0 
        for grad in grads:
            if grad != None: grad_sum += torch.sum(grad)
        return grad_sum

    def train_net(self, net, net_optim, net_input, net_labels):
        
        net_input = net(net_input)
        net_prob = self.softmaxlayer(net_input)
        net_loss = self.nll_loss(torch.log(net_prob+1e-16), net_labels)

        # Backprop
        net_optim.optimizer.zero_grad()
        net_loss.backward()
        _, grad_norm, _ = net_optim.step_all()

#         d_prec1 = accuracy(d_prob.data, self.sensitive_label.data)
#         d_prec5 = accuracy(d_prob.data, self.sensitive_label.data, topk=(int(np.min([5, sensitive_classes])),))

        return net_loss, grad_norm

    

    def arl_train(self, criterion, d_loc=None, privacy_flag=True, public_flag = False, arl_setting='maxent-arl', debug_steps=100, epoch=-1, verbose=False):
        ''' ===================================================================================
        Main Algorithm:
            1. Train encoder and decoder with discriminator loss
            2. Train discriminator
        ========================================================================================'''
        self.net.train(True)
        if privacy_flag: self.discriminator.train(True)
        if public_flag: self.public.train(True)
        self.meters_reset()

        for i, data in enumerate(self.train_loader):

            # Prepare data
            if i%10 == 0 and i: print(end='.')
            self.images.resize_(data[0].size()).copy_(data[0])
            self.boxes.resize_(data[1].size()).copy_(data[1])
            self.labels.resize_(data[2].size()).copy_(data[2])

            # 1. Train Encoder + Predictor =============
            confidence, locations, z = self.net(self.images, d_loc)
            class_loss, reg_loss = criterion(confidence, locations, self.labels, self.boxes)
            ssd_loss = class_loss + reg_loss
                    
            # Other public task other than object detection
            if public_flag:
                public_labels = data[4] if len(data) == 5 else self.online_find_labels(self.labels, self.public_classes)
                self.public_labels.resize_(public_labels.size()).copy_(public_labels)     
            
                self.public_optimizer.optimizer.zero_grad()
                p_output = self.public(z)
                p_prob = self.softmaxlayer(p_output)
                p_loss = self.nll_loss(torch.log(p_prob+1e-16), self.public_labels)    
                
            # Loss =====================================
            if privacy_flag:
                sensitive_labels = data[3] if len(data) == 4 else self.online_find_labels(self.labels, self.sensitive_classes)
                self.sensitive_labels.resize_(sensitive_labels.size()).copy_(sensitive_labels)     
                
                self.discriminator_optimizer.optimizer.zero_grad()
                d_output = self.discriminator(z)
                d_prob = self.softmaxlayer(d_output)
                
                # Calculate for private task entropy/log liklihood
                if arl_setting == 'maxent-arl':
                    entropy_loss = -self.entropy_loss(d_prob)
                elif arl_setting == 'ml-arl':
                    entropy_loss = self.nll_loss(torch.log(d_prob+1e-16), self.sensitive_labels)
                else:
                    raise Exception("Invalid ARL.")
                s_loss = -entropy_loss
            
            # Combined loss to update SSD (+ public task)
            if privacy_flag:
                if public_flag:
                    loss = (1-self.alpha)*(p_loss) + self.alpha*s_loss + ssd_loss
                else:
                    loss = (1-self.alpha)*(ssd_loss) + self.alpha*s_loss
            else:
                loss = ssd_loss

            # Backprop ==================================
            self.optimizer.optimizer.zero_grad()  
            loss.backward()
            if public_flag: _, _, _ = self.public_optimizer.step_all()                
            enc_grad_norm, self.grad_norm, shrinkage = self.optimizer.step_all()
                
            # 2. Train Discriminator ====================
            if privacy_flag: 
                d_loss, _ = self.train_net(net = self.discriminator, 
                                           net_optim = self.discriminator_optimizer,
                                           net_input =z.detach(), 
                                           net_labels = self.sensitive_labels)    
        
            # 3. Save and Show Results ==================
            self.running_loss += loss.item() / debug_steps
            self.running_ssd_loss += ssd_loss.item() / debug_steps
            self.running_regression_loss += reg_loss.item()  / debug_steps
            self.running_classification_loss += class_loss.item() / debug_steps
            self.report_grad_norm += self.grad_norm / debug_steps
            if privacy_flag: 
                self.running_discriminator_loss += d_loss.item() / debug_steps
                self.running_discriminator_entropy += entropy_loss.item() /debug_steps
                
            if public_flag:
                self.running_public_loss += p_loss.item() / debug_steps
            
            #  Show the average results
            if i and i % debug_steps == 0:
                self.log(f"Epoch:{epoch:02d}, Step:{i}, " +
                             f"Loss:{self.running_loss:.4f}, " +
                             f"SSD Loss:{self.running_ssd_loss:7.4f} [" +
                             f"Reg:{self.running_regression_loss:.4f}, " +
                             f"Class:{self.running_classification_loss:.4f}], " + 
                             f"ARL Loss:{self.running_discriminator_entropy:.4f}, " + 
                             f"D Loss:{self.running_discriminator_loss:.4f}, " + 
                             f"P Loss:{self.running_public_loss:.4f}, " +
                             f"Grad Norm:{self.report_grad_norm:.4f}"
                            )
                self.meters_reset()

                # for param in net.classification_headers.parameters():
                #     print(f"Classification:{param[0][0]}")
                #     break                
    
    def test(self, criterion, d_loc, privacy_flag = False, public_flag=False):
        
        self.net.eval()
        if privacy_flag: self.discriminator.eval()
        if public_flag: self.public.eval()
        self.meters_reset()
        num = 0
        self.net.test = True

        for i, data in enumerate(self.test_loader):

            num += 1
            if i%10 == 0: print(end='.')
            self.images.resize_(data[0].size()).copy_(data[0])
            self.boxes.resize_(data[1].size()).copy_(data[1])
            self.labels.resize_(data[2].size()).copy_(data[2])

            # Sensitive label
            
            #  Feedforward
            with torch.no_grad():
                confidence, locations, z = self.net(self.images, d_loc)
                class_loss, reg_loss = criterion(confidence, locations, self.labels, self.boxes)
                loss = class_loss + reg_loss
                
                if privacy_flag:
                    sensitive_labels = data[3] if len(data) == 4 else self.online_find_labels(self.labels, self.sensitive_classes)
                    self.sensitive_labels.resize_(sensitive_labels.size()).copy_(sensitive_labels)   
                    d_output = self.discriminator(z)
                    d_prob = self.softmaxlayer(d_output)
                    entropy_loss = -self.entropy_loss(d_prob)
                    d_loss = self.nll_loss(torch.log(d_prob+1e-16), self.sensitive_labels)
                    
                if public_flag:
                    public_labels = data[4] if len(data) == 5 else self.online_find_labels(self.labels, self.public_classes)
                    self.public_labels.resize_(public_labels.size()).copy_(public_labels)     
                    z = self.public(z)
                    p_prob = self.softmaxlayer(z)
                    p_entropy_loss = -self.entropy_loss(p_prob)
                    p_loss = self.nll_loss(torch.log(p_prob+1e-16), self.public_labels)

            #  Save numerical results
            self.running_loss += loss.item()
            self.running_regression_loss += reg_loss.item()
            self.running_classification_loss += class_loss.item()
            if privacy_flag:
                self.running_discriminator_loss += d_loss.item()
                self.running_discriminator_entropy += entropy_loss.item()
            if public_flag:
                self.running_public_loss += p_loss.item()

        self.net.test = False

        return (self.running_loss / num, self.running_regression_loss / num, self.running_classification_loss / num, self.running_discriminator_loss / num, 
                self.running_discriminator_entropy / num, self.running_public_loss / num)
    
    
    def arl_attack(self, criterion, attacker_loc, attack_criterion=None, debug_steps=40, epoch=-1, verbose=False, public_flag=False):
        ''' ===================================================================================
        Main Algorithm:
            1. Train encoder and decoder with discriminator loss
            2. Train discriminator
        ======================================================================================== '''
        self.net.eval()
        self.discriminator.train(True)
        self.meters_reset()

        for i, data in enumerate(self.train_loader):

            # Prepare data
            if i%10 == 0 and i: print(end='.')
            self.images.resize_(data[0].size()).copy_(data[0])
            self.boxes.resize_(data[1].size()).copy_(data[1])
            self.labels.resize_(data[2].size()).copy_(data[2])
            sensitive_labels = data[3] if len(data) == 4 else self.online_find_labels(self.labels, self.sensitive_classes)
            self.sensitive_labels.resize_(sensitive_labels.size()).copy_(sensitive_labels)   

            # 1. Train Encoder + Predictor =============
            with torch.no_grad():
                confidence, locations, z = self.net(self.images, attacker_loc)
                class_loss, reg_loss = criterion(confidence, locations, self.labels, self.boxes)
                ssd_loss = class_loss + reg_loss
                
                
            d_loss, self.grad_norm = self.train_net(net=self.discriminator, 
                                                    net_optim=self.discriminator_optimizer,
                                                    net_input=z.detach(), 
                                                    net_labels=self.sensitive_labels)
            
#             if public_flag:
#                 public_labels = data[4] if len(data) == 5 else self.online_find_labels(self.labels, self.public_classes)
#                 self.public_labels.resize_(public_labels.size()).copy_(public_labels)     
            
#                 self.public_optimizer.optimizer.zero_grad()
#                 p_output = self.public(z)
#                 p_prob = self.softmaxlayer(p_output)
#                 p_loss = self.nll_loss(torch.log(p_prob+1e-16), self.public_labels)   

            # 3. Save and Show Results ==================
            self.running_loss += d_loss.item() / debug_steps
            self.running_ssd_loss += ssd_loss.item() / debug_steps
            self.running_regression_loss += reg_loss.item()  / debug_steps
            self.running_classification_loss += class_loss.item() / debug_steps
            self.report_grad_norm += self.grad_norm / debug_steps
            
            #  Show the average results
            if i and i % debug_steps == 0:
                self.log(f"Epoch:{epoch:02d}, Step:{i:3d}, " +
                             f"Attacker Loss:{self.running_loss:.4f}, " +
                             f"SSD Loss:{self.running_ssd_loss:7.4f} [" +
                             f"Reg:{self.running_regression_loss:.4f}, " +
                             f"Class:{self.running_classification_loss:.4f}], " + 
                             f"Grad Norm:{self.report_grad_norm:.4f}"
                            )
                self.meters_reset()

                # for param in net.classification_headers.parameters():
                #     print(f"Classification:{param[0][0]}")
                #     break                
        
    def test_attack(self, criterion, attacker_loc, privacy_flag = True):
        
        self.net.eval()
        self.discriminator.eval()
        self.meters_reset()
        num = 0
        self.net.test = True

        for i, data in enumerate(self.test_loader):

            num += 1
            if i%10 == 0: print(end='.')
            self.images.resize_(data[0].size()).copy_(data[0])
            self.boxes.resize_(data[1].size()).copy_(data[1])
            self.labels.resize_(data[2].size()).copy_(data[2])

            # Sensitive label
            sensitive_labels = data[3] if len(data) == 4 else self.online_find_labels(self.labels, self.sensitive_classes)
            self.sensitive_labels.resize_(sensitive_labels.size()).copy_(sensitive_labels)   
            
            #  Feedforward
            with torch.no_grad():
                confidence, locations, z = self.net(self.images, attacker_loc)
                class_loss, reg_loss = criterion(confidence, locations, self.labels, self.boxes)
                loss = class_loss + reg_loss
                
                d_output = self.discriminator(z)
                d_prob = self.softmaxlayer(d_output)
                d_loss = self.nll_loss(torch.log(d_prob+1e-16), self.sensitive_labels)
                
                entropy_loss = -self.entropy_loss(d_prob)
                s_loss = -entropy_loss 

                #  Loss


            #  Save numerical results
            self.running_loss += loss.item()
            self.running_regression_loss += reg_loss.item()
            self.running_classification_loss += class_loss.item()
            self.running_discriminator_loss += d_loss.item()
            self.running_discriminator_entropy += entropy_loss.item()

                 

        self.net.test = False

        return self.running_loss/num, self.running_regression_loss/num, self.running_classification_loss/num, self.running_discriminator_loss/num, self.running_discriminator_entropy/num
    

## main

Checkpoint Folder: https://drive.google.com/drive/u/1/folders/1trRWtHV_io4EDScepki1Q1RS_YewKdTA

Argparse in Jupyternotebook: https://zhuanlan.zhihu.com/p/145720581

Restore: https://pytorch.org/tutorials/beginner/saving_loading_models.html


In [17]:
def create_dir():
    experiment_time = 1
    testing_dir = f"/home/hjchris/research/mitigating/chris/pascal_ckpts/{today}/{today}-{experiment_time}"
    while (os.path.exists(testing_dir) and len([name for name in os.listdir(testing_dir) if os.path.isfile(os.path.join(testing_dir, name))]) != 0):
        experiment_time += 1
        testing_dir = f"/home/hjchris/research/mitigating/chris/pascal_ckpts/{today}/{today}-{experiment_time}"
    
    return testing_dir, experiment_time

In [18]:
class TrainingSetup:
    def __init__(self, pretrained_encoder= None, log=print):


        self.log=log
        # network hyperparameters
        self.learning_rate = 1e-2
        self.discriminator_lr = 1e-4
        self.public_lr = 1e-4
        self.encoder_lr = 1e-4
        self.base_net_lr =  self.learning_rate
        self.extra_layers_lr = self.learning_rate
        self.num_classes = 21
        
        # ARL setting
        self.privacy_flag = True
        self.public_flag = False
        self.attack_flag = False
        self.save = True
        self.saving_more = True
        self.continue_training = False

        # encoder setting
        self.public_headers = None
        self.public_optimizer = None
        self.public_scheduler = None
        
        self.discriminator_headers = None
        self.discriminator_optimizer = None
        self.discriminator_scheduler = None
        
        self.encoder_flag=False
        self.pretrained_encoder = pretrained_encoder

        # network general setting        
        self.last_epoch = -1
        self.num_epochs = 201
        self.loading_net = "basenet"
        self.scheduler_type = 'cosine'

        # optimizer setting
        self.opt_type = 'sgd'
        self.max_grad_norm = 6
        self.verbose = False
        self.t_max = 200
        self.momentum = 0.9
        self.weight_decay = 0.0005
        self.min_loss = -10000.0
        
        self.timer = Timer()

        if torch.cuda.is_available():
            torch.backends.cudnn.benchmark = True
            train_log.info("Use cuda:0")
    
    def set_directory(self):
        save_path, experiment_time = create_dir()
        self.log(f"="*45 + f" Training Session {today}-{experiment_time} " + f"="*45)
        
        if not os.path.exists(save_path): os.mkdir(save_path)
        self.checkpoint_folder = save_path
        
        if self.save: 
            self.log(f"Model will be stored at {self.checkpoint_folder}")
    
    def build_discriminator(self, d_loc):
        create_discriminator = Create_Discriminator()
        return create_discriminator.build(d_loc)
    
    def build_network(self):
        self.log("Build SSD.")
        create_net = create_mobilenetv2_ssd_lite(num_classes = self.num_classes, width_mult = 1.0)
        return create_net.create_SSD()

    def build_init(self, load_path):
        timer = Timer()
        timer.start("Load Model")
        
        # load mbovilenetv2 base parameters
        if "mb2-imagenet-71_8.pth" in load_path:
            self.log(f"Initialize SSD basenet from: {load_path}")
            self.net.init_base_net(load_path)
            self.log(f'Took {timer.end("Load Model"):.2f} seconds to load mobilenet into the model.')

        else:
            self.load_checkpoint(load_path)
        
    def build_rest(self):
        if not self.save: self.log(f"Not Saving model.")

    def build_scheduler(self, optimizer, name = 'SSD', privacy_flag = False):
        
        if self.scheduler_type == "multi-step":
            self.log("Use MultiStepLR scheduler.")
            milestones = [int(v.strip()) for v in "80,100".split(",")]
            return MultiStepLR(optimizer, milestones=milestones, gamma=0.1, last_epoch=last_epoch)
        else:
            self.log(name + f" build [{self.scheduler_type}] CosineAnnealingLR scheduler. Max iter.={self.t_max}, last epoch={self.last_epoch}.")
            return CosineAnnealingLR(optimizer, self.t_max, last_epoch=self.last_epoch)

    def build_criterion(self):
        self.log(f"Build MultiboxLoss. Use mobilnetv1 SSD configuration.")
        config = mobilenetv1_ssd_config
        return MultiboxLoss(config.priors, iou_threshold=0.5, neg_pos_ratio=3, center_variance=0.1, size_variance=0.2, device=DEVICE)

    def build_optim(self, parameters=None, name='SSD', learning_rate=1e-4, privacy_flag = False, opt_type = 'sgd'):
        self.log(name +  f" build {opt_type} optimizer in Optim, momentum={self.momentum}, " + 
                     f"weight_decay={self.weight_decay}, " + 
                     f"max gradient norm={self.max_grad_norm}.")   
        params=[]
        
        if not privacy_flag:
            self.log("Use ssd parameters.")
            params = [{'params': self.net.base_net.parameters(),'lr': self.base_net_lr},
                      {'params': itertools.chain(self.net.source_layer_add_ons.parameters(), self.net.extras.parameters()),'lr': self.extra_layers_lr},
                      {'params': itertools.chain(self.net.regression_headers.parameters(), self.net.classification_headers.parameters())}
                     ]

            # Encoder Parameters
            if self.net.encoder_headers:
                self.log(f"Use extra encoder parameters.")
                params.append({'params': self.net.encoder_headers.parameters(),'lr': self.encoder_lr})
                encoder_params = self.net.encoder_headers.parameters()
            else:
                encoder_params = self.net.base_net.parameters()
                
        else:
            params.append({'params': parameters,'lr': learning_rate})
            encoder_params = []

        return Optim(params = params, 
                     method = opt_type, 
                     enc_params = encoder_params,
                     max_grad_norm = self.max_grad_norm, 
                     lr=self.learning_rate,
                     sgd_momentum=self.momentum,
                     sgd_weight_decay=self.weight_decay,
                     privacy_flag=privacy_flag
                     )
    
    # ==========================================================================
    #                               Main Logic
    # ==========================================================================
    def build(self, load_path, privacy_flag, public_flag, d_loc, opt_type='sgd', discriminator_opt_type='adam', is_attack=False):
        
        self.public_flag = public_flag
        self.privacy_flag = privacy_flag
        self.set_directory()

        # create SSD [encoder and decoder]
        self.net = self.build_network()
        self.net.init()
        self.log("Xavier Initialization for SSD.")
        self.build_init(load_path)
        self.optimizer = self.build_optim(privacy_flag = False, opt_type=opt_type)
        self.scheduler = self.build_scheduler(self.optimizer.optimizer, privacy_flag=False)
        
        # public task
        if self.public_flag:
            self.public_flag = public_flag
            self.public_headers = self.build_discriminator(d_loc)
            self.public_optimizer = self.build_optim(name='Public',
                                                     parameters = self.public_headers.parameters(),
                                                     learning_rate = self.public_lr,
                                                     privacy_flag = True, 
                                                     opt_type = discriminator_opt_type)
            self.public_scheduler = self.build_scheduler(name='Public',
                                                         optimizer=self.public_optimizer.optimizer, 
                                                         privacy_flag=True)
            self.public_headers.to(DEVICE)
        
        # private task
        if self.privacy_flag:   
            build_name = 'Attacker' if is_attack else 'Discrimiantor'
            self.discriminator_headers = self.build_discriminator(d_loc)
            self.discriminator_optimizer = self.build_optim(name=build_name,
                                                            parameters = self.discriminator_headers.parameters(),
                                                            learning_rate = self.discriminator_lr,
                                                            privacy_flag=True, 
                                                            opt_type=discriminator_opt_type)
            self.discriminator_scheduler = self.build_scheduler(name=build_name,
                                                                optimizer = self.discriminator_optimizer.optimizer, 
                                                                privacy_flag=True)
            self.discriminator_headers.to(DEVICE)
        
        # loss function
        self.criterion = self.build_criterion()
        
        # loading parameters
        self.build_rest()
        self.net.to(DEVICE)
    # ==========================================================================
    #                             Main Logic End
    # ==========================================================================
    
    def load_checkpoint(self, load_path):
        self.log(f"Load SSD from: {load_path}")
        self.checkpoint = torch.load(load_path, map_location=lambda storage, loc: storage)
        self.net.init_from_pretrained_ssd(load_path, log=train_log.info)  
        
        self.privacy_flag = self.checkpoint['privacy_flag']
        self.max_grad_norm = self.checkpoint['max_grad_norm']
        self.log(f"privacy_flag={self.privacy_flag}, max_grad_norm={self.max_grad_norm}")
        
        if self.checkpoint['privacy_flag']:
            self.alpha = self.checkpoint['alpha']
            self.sensitive_classes = self.checkpoint['sensitive_classes']
            self.discriminator_loc = self.checkpoint['discriminator_loc']
            self.log(f"alpha={self.alpha[0]}, sensitive_classes={self.sensitive_classes}, discriminator_loc={self.discriminator_loc}")
        else:
            self.alpha = 0.0
            self.sensitive_classes = [15]
            
    
    def attack(self, train_loader, val_loader, attacker_loc, verbose=False, val_epoch=5, skip_train=False, previous_model=''): 
        ''' ====================================================================
        ARL Attack. 
        The discriminator is used as the attacker.
        
        
        ==================================================================== '''
        self.log(f"train loader={len(train_loader)}, validation loader={len(val_loader)}")    
        self.log(f"Start attack the model.")
        attack_model = previous_model
        
        nets = [self.net, self.discriminator_headers, self.public_headers]
        optimizers = [None, self.discriminator_optimizer, None]
        schedulers = [None, self.discriminator_scheduler, None]
        
        train_net = Maxtrain_Network(nets, optimizers, train_loader, val_loader, self.alpha, self.sensitive_classes, log=self.log)
        
        for epoch in range(self.last_epoch + 1, self.num_epochs):
                                     
            if not skip_train:
                train_net.arl_attack(self.criterion, 
                                     attacker_loc, 
                                     debug_steps=40, 
                                     epoch=epoch)
                for scdlr in schedulers:
                    scdlr.step() if scdlr else None
            
            if epoch % val_epoch == 0 or epoch == self.num_epochs - 1 :                
                val_loss, val_reg_loss, val_class_loss, val_loss_dis, val_ent = train_net.test_attack(self.criterion, attacker_loc, self.privacy_flag)
                self.log(f"Epoch:{epoch:02d}, " +
                             f"Validation Loss:{val_loss + val_loss_dis:.4f}/{val_loss:.4f}, " +
                             f"Reg Loss:{val_reg_loss:.4f}, " +
                             f"Class Loss:{val_class_loss:.4f}, " +
                             f"D Loss:{val_loss_dis:.4f}, " + 
                             f"D Entropy:{val_ent:.4f}"
                             )                    
                #  Save
                if self.save:
                    model_name = f"{'mb2-ssd-lite'}-Attack-Ep{epoch:03d}-FM{attacker_loc}-Loss-{val_loss:.4f}+{val_loss_dis:.4f}-Ent-{val_ent:.4f}.pth"
                    model_path = os.path.join(self.checkpoint_folder, model_name)
                    self.net.save_attack(privacy_flag=self.privacy_flag, 
                                         public_flag=self.public_flag,
                                         alpha = self.alpha, 
                                         max_grad_norm = self.max_grad_norm, 
                                         epoch = epoch,
                                         optimizers = optimizers,
                                         schedulers = schedulers, 
                                         ssd_loss = val_loss, 
                                         sensitive_classes = self.sensitive_classes,
                                         entropy = val_ent,
                                         attacker = self.discriminator_headers,
                                         attacker_loss = val_loss_dis, 
                                         attacker_loc = attacker_loc,
                                         model_path = model_path,
                                         name = attack_model
                                        )
                    self.log(f"Save models at: ~{model_path[52:]}")
                    if os.path.exists(previous_model): os.remove(previous_model)
                    previous_model = model_path
    
    
    def train(self, train_loader, val_loader, d_loc = 14, public_classes = [7], sensitive_classes=[15], alpha=1.0, verbose=False, val_epoch=5, 
              skip_train=False, previous_model='', arl_setting=None):
        ''' ======================================================================
        ARL Training
        
        ========================================================================== '''  

        if self.privacy_flag:
            self.log(f"Discriminator location={d_loc}, Alpha={alpha}, Sensitive classes={sensitive_classes}, Arl_setting={arl_setting}")
        if self.public_flag:
            self.log(f"Public classes={public_classes}")            

        self.log(f"train loader={len(train_loader)}, validation loader={len(val_loader)}")    
        self.log(f"Start training from epoch {self.last_epoch + 1}.") 
        
        nets = [self.net, self.discriminator_headers, self.public_headers]
        optimizers = [self.optimizer, self.discriminator_optimizer, self.public_optimizer]
        schedulers = [self.scheduler, self.discriminator_scheduler, self.public_scheduler]
        
        if self.privacy_flag:
            alpha = torch.tensor([alpha*1.0], requires_grad=True)

        train_net = Maxtrain_Network(nets, optimizers, train_loader, val_loader, alpha, sensitive_classes, public_classes, log=self.log)

        for epoch in range(self.last_epoch + 1, self.num_epochs):       
            
            if not skip_train:
                train_net.arl_train(self.criterion, 
                                    privacy_flag = self.privacy_flag, 
                                    public_flag  = self.public_flag, 
                                    d_loc=d_loc,
                                    debug_steps=100, 
                                    epoch=epoch, 
                                    verbose=verbose,
                                    arl_setting = arl_setting,
                                   )
                
                for scdlr in schedulers:
                    scdlr.step() if scdlr else None

            # Evaluation
            if epoch % val_epoch == 0 or epoch == self.num_epochs - 1 :                

                val_loss, val_reg_loss, val_class_loss, val_d_loss, val_d_ent, val_p_loss = train_net.test(self.criterion,  
                                                                                                           d_loc=d_loc,
                                                                                                           privacy_flag=self.privacy_flag,
                                                                                                           public_flag  = self.public_flag,
                                                                                                          )
                self.log(f"Epoch:{epoch:02d}, " +
                             f"Validation Loss:{val_loss + val_d_ent:.4f} [" +
                             f"Reg Loss:{val_reg_loss:.4f}, " +
                             f"Class Loss:{val_class_loss:.4f}], " +
                             f"Entropy:{val_d_ent:.4f}, " +
                             f"D Loss:{val_d_loss:.4f}, " +
                             f"P Loss:{val_p_loss:.4f}"
                             ) 
                
                if self.save:
                    model_name = f"{'mb2-ssd-lite'}-Ep{epoch:03d}"
                    if self.privacy_flag:
                        model_name += f"-FM{d_loc}-A{alpha.data[0]:.1f}"
                    model_name += f"-Loss-{val_loss:.4f}"
                    if self.privacy_flag:
                        model_name += f"+{val_d_ent:.4f}-DL-{val_d_loss:.4f}"
                    if self.public_flag:
                        model_name += f"-PL-{val_p_loss:.4f}"
                    model_name += f".pth"
                        
                        
                    model_path = os.path.join(self.checkpoint_folder, model_name)
                    
                    self.net.save(privacy_flag = self.privacy_flag, 
                                  public_flag = self.public_flag,
                                  alpha = alpha, 
                                  max_grad_norm = self.max_grad_norm, 
                                  epoch = epoch, 
                                  optimizers = optimizers, 
                                  schedulers = schedulers, 
                                  val_loss = val_loss, 
                                  val_d_ent = val_d_ent, 
                                  val_d_loss = val_d_loss,
                                  sensitive_classes = sensitive_classes,
                                  discriminator_loc = d_loc,
                                  discriminator= self.discriminator_headers,
                                  model_path = model_path,
                                  public=self.public_headers,
                                  val_p_loss = val_p_loss,
                                  public_classes = public_classes,
                                  arl_setting = arl_setting
                                 )
                    
                    self.log(f"Save models at: ~{model_path[52:]}")
                    if os.path.exists(previous_model): os.remove(previous_model)
                    previous_model = model_path
                    

In [19]:
def find_ssd_file(checkpoint_folder):
    
    filenames=[]
    for filename in os.listdir(checkpoint_folder):
        if "pth" in filename:
            filenames.append(filename)
            
    ssd_path = checkpoint_folder + filenames[0]
    epoch = filenames[0].split("Ep")[1][:3]
    eval_path = checkpoint_folder + f"evaluation_{epoch}/"
    
    if not os.path.exists(eval_path):os.mkdir(eval_path)
        
    print(f"ssd path: {ssd_path}")
    print(f"eval path: {eval_path}")
    return ssd_path, eval_path

In [20]:
import datetime


def write_doc(name, mode, dictionary, method=None, message=None):
    time = datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
    with open("chris/pascal_ckpts/document.txt", 'a') as file:
        file.write(f"{time} - File={name},\tTask={mode},\t")  
        for item in dictionary.items():
                file.write(f"{item[0]}={item[1]},\t ")
                if item[0] == 'privacy_flag' and item[1] == False: break
                if item[0] == 'public_flag' and item[1] == False: break
                
        if message:
                file.write("msg=(" + str(message) + ")\t")
        file.write("end\n")
        
def write_doc_end(name, mode):
    time = datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
    with open("chris/pascal_ckpts/document.txt", 'a') as file:
        file.write(f"{time} - File {name}   {mode} ends.\n")
            

## run

1:**aeroplane**
2:**bicycle**
3:**bird**
4:**boat**
5:**bottle**
6:**bus**
7:**car**
8:**cat**
9:**chair**
10:**cow**
11:**diningtable**
12:**dog**
13:**horse**
14:**motorbike**
15:**person**
16:**pottedplant**
17:**sheep**
18:**sofa**
19:**train**
20:**tvmonitor**

In [ ]:
# dataload_folder = '/home/hjchris/data/Pascal_VOC'
root_path = f"/home/hjchris/data"
data_prep = Data_Preprocessing(log=train_log.info)
data_prep.load_online(root_path)


message = None
public_flag_set = [False]
arl_setting = ['ml-arl']
arl_set = ['maxent-arl', 'ml-arl']
locations = [18]

attack_dates = ["1221","1221","1221","1221","1221","1221","1221","1221","1221","1221","1221","1221","1221"]
attack_nums = [1, 1, 1, 1, 1, 2, 2, 2, 2, 2 ]
attack_locs = [14, 18, 19, 20, 21, 14, 18, 19, 20, 21]


todays_task_is_train = True

if not todays_task_is_train:
    print("Attacking:")
    for att in range(min(len(attack_dates), len(attack_nums))):
        attack_date = attack_dates[att]
        attack_num = attack_nums[att]
        attack_loc = attack_locs[att]
        print(f"{attack_date}_21/{attack_date}_21-{attack_num}-loc-{attack_loc}")
    print()


if todays_task_is_train:
    for pb in public_flag_set:
        for arl_setting in arl_set:
            for d_loc in locations:
                train_dict ={'privacy_flag':True,
                             'method': arl_setting,
                             'alpha': 0.7,
                             'd_loc': d_loc,
                             'd_type': "A",
                             'sensitive_classes':[15],
                             'public_flag':pb,
                             'public_classes':[7],
                             'p_type': "A",
                            }

                arl_model = TrainingSetup(data_prep, log=train_log.info)

                for run in range(1):

                    gc.collect()
                    torch.cuda.empty_cache()

                    basenet_path = os.path.join(f"{root_path}/Pascal_VOC", f"mb2-imagenet-71_8.pth")
                    arl_model.build(load_path = basenet_path,
                                    d_loc = train_dict['d_loc'],
                                    public_flag = train_dict['public_flag'],
                                    privacy_flag = train_dict['privacy_flag']
                                    )

                    _, experiment_time = create_dir()
                    write_doc(f"{today}-{experiment_time}", mode="train ", dictionary=train_dict, method=arl_setting, message=message)

                    arl_model.train(data_prep.train_loader, 
                                    data_prep.val_loader,
                                    arl_setting = train_dict['method'],
                                    alpha = train_dict['alpha'],
                                    d_loc = train_dict['d_loc'],
                                    verbose = True, 
                                    public_classes = train_dict['public_classes'],
                                    sensitive_classes = train_dict['sensitive_classes'], 
                                    val_epoch = 5,
                                    skip_train = False
                                    )
                    write_doc_end(f"{today}-{experiment_time}", mode="train")

                
else:
    for att in range(min(len(attack_dates), len(attack_nums))):

        attack_date = attack_dates[att]
        attack_num = attack_nums[att]
        attack_loc = attack_locs[att]
        
        attack_root = '/home/hjchris/research/mitigating/chris/pascal_ckpts/'
        attack_path, _ = find_ssd_file(attack_root + f"{attack_date}_21/{attack_date}_21-{attack_num}/")

        attack_dict ={'attack_model':f"{attack_date}_21-{attack_num}", 
                      'attacker_loc': attack_loc,
                      'a_type': "A"
                     }

        arl_model = TrainingSetup(data_prep, log=train_log.info)

        gc.collect()
        torch.cuda.empty_cache()

        arl_model.build(load_path = attack_path, 
                        d_loc = attack_dict['attacker_loc'],
                        public_flag = False, 
                        privacy_flag = True, 
                        is_attack=True
                       )

        _, experiment_time = create_dir()
        write_doc(f"{today}-{experiment_time}", mode="attack", dictionary=attack_dict)

        arl_model.attack(data_prep.train_loader_attack, 
                         data_prep.val_loader_attack,
                         attacker_loc = attack_dict['attacker_loc'],
                         verbose=True, 
                         val_epoch=5,
                         previous_model = attack_dict['attack_model'],
                         skip_train=False
                       )
        
        write_doc_end(f"{today}-{experiment_time}", mode="attack")

2022-01-02 10:59:09,466 - train - INFO - Using Original VOCdataset functions? Training:True, Testing:True.
2022-01-02 10:59:09,469 - train - INFO - Prepare training datasets
No labels file, using default VOC classes.
No labels file, using default VOC classes.
No labels file, using default VOC classes.
No labels file, using default VOC classes.
2022-01-02 10:59:09,499 - train - INFO - Stored labels into file.
2022-01-02 10:59:09,501 - train - INFO - Train data: 11584, loader: 362, batch_size=32, shuffle=True
2022-01-02 10:59:09,502 - train - INFO - Train attack data : 4967, loader: 156, batch_size=32, shuffle=True
2022-01-02 10:59:09,503 - train - INFO - Prepare Validation datasets.
No labels file, using default VOC classes.
No labels file, using default VOC classes.
2022-01-02 10:59:09,514 - train - INFO - Valid data: 3466, loader: 109, batch_size=32, shuffle=False.
2022-01-02 10:59:09,515 - train - INFO - Valid attack data: 1486, loader:47, batch_size=32, shuffle=False
2022-01-02 10:5

Trainformation Explanation: https://hellozhaozheng.github.io/z_post/PyTorch-SSD-%E6%95%B0%E6%8D%AE%E5%A4%84%E7%90%86/


# mAP

https://drive.google.com/drive/u/1/folders/1trRWtHV_io4EDScepki1Q1RS_YewKdTA

In [ ]:
from vision.utils import box_utils
from vision.ssd.data_preprocessing import PredictionTransform
from vision.utils.misc import Timer
    

class Predictor:
    def __init__(self, net, size, mean=0.0, std=1.0, nms_method=None,
                 iou_threshold=0.45, filter_threshold=0.01, candidate_size=200, sigma=0.5, device=None):
        self.net = net
        self.transform = PredictionTransform(size, mean, std)
        self.iou_threshold = iou_threshold
        self.filter_threshold = filter_threshold
        self.candidate_size = candidate_size
        self.nms_method = nms_method

        self.sigma = sigma
        if device:
            self.device = device
        else:
            self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        self.net.to(self.device)
        self.net.eval()

        self.timer = Timer()
        
    def image_transform(self, image):
        image = self.transform(image)
        image = image.unsqueeze(0)
        image = image.to(self.device)
        
        return image

    def predict(self, image, cnt, discriminator_loc, top_k=-1, prob_threshold=None):

        #  Transforme the image. The input is only one image
        cpu_device = torch.device("cpu")
        height, width, _ = image.shape
        image = self.transform(image)
        images = image.unsqueeze(0)
        images = images.to(self.device)
        
        #  Feedfoward the image into the network
        with torch.no_grad():
            self.timer.start()
            scores, boxes, _ = self.net(images, discriminator_loc)
            if cnt % 100 == 0:
                print("Inference time: {:4f} seconds.".format(self.timer.end()), end='   ')

        #  There is only one image. Because the output of the network is a batch of size of 1, get rid of the batch bracket
        boxes = boxes[0]  # size = [1,3000,4]
        scores = scores[0]  # size = [1,3000,21]
        if not prob_threshold:
            prob_threshold = self.filter_threshold

        # this version of nms is slower on GPU, so we move data to CPU.
        boxes = boxes.to(cpu_device)
        scores = scores.to(cpu_device)
        picked_box_probs = []
        picked_labels = []

        #  Inference class by class. 
        for class_index in range(1, scores.size(1)):
            
            #  Confidence
            probs = scores[:, class_index]
            mask = probs > prob_threshold # select the probability that is larger than threshold
            probs = probs[mask]
            if probs.size(0) == 0:
                continue
            
            #  Locations
            subset_boxes = boxes[mask, :] # select the box that has the label with probability larger than threshold. subset_boxes = [n,4]
            box_probs = torch.cat([subset_boxes, probs.reshape(-1, 1)], dim=1)      # box_probs = [n, 4+1]
            
            """TO BE UNDERSTOOD: WHAT DOES NMS DO?"""
            box_probs = box_utils.nms(box_probs, self.nms_method,
                                      score_threshold=prob_threshold,
                                      iou_threshold=self.iou_threshold,
                                      sigma=self.sigma,
                                      top_k=top_k,
                                      candidate_size=self.candidate_size)
            
            #  append the results
            picked_box_probs.append(box_probs)
            picked_labels.extend([class_index] * box_probs.size(0))

        #  If the picked boxes are not empty, the boxes are transformed to the absolute coordinate        
        if not picked_box_probs:
            return torch.tensor([]), torch.tensor([]), torch.tensor([])
        picked_box_probs = torch.cat(picked_box_probs)
        picked_box_probs[:, 0] *= width
        picked_box_probs[:, 1] *= height
        picked_box_probs[:, 2] *= width
        picked_box_probs[:, 3] *= height
        return picked_box_probs[:, :4], torch.tensor(picked_labels), picked_box_probs[:, 4]


In [ ]:
from vision.utils import box_utils, measurements

class Inference_ARL:
    
    def __init__(self, to_log=True, log=print):
        self.dataset = VOCDataset('/home/hjchris/data/Pascal_VOC/VOC2007/', is_test=True) # just 2007 test
        self.true_case_stat, self.all_gb_boxes, self.all_difficult_cases = self.group_annotation_by_class(self.dataset)
        self.iou_threshold = 0.5
        self.use_2007_metric = True
        self.to_log = to_log
        self.log = log
        
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.use_cuda = torch.cuda.is_available()
        self.images = torch.zeros(0, 0, 0)
        self.boxes = torch.zeros(0, 0, 0)
        self.labels = torch.zeros(0, dtype=torch.long)
        self.moduler_labels = torch.zeros(0, dtype=torch.long)
        
        self.entropy_loss = EntropyLoss()
        self.softmaxlayer = nn.Softmax(dim=1)
        self.logsoftmax = nn.LogSoftmax(dim=1)
        self.nll_loss = nn.NLLLoss()
        self.mse_loss = nn.MSELoss()

        if self.use_cuda:
            self.images=self.images.cuda()
            self.boxes=self.boxes.cuda()
            self.labels=self.labels.cuda()
            self.moduler_labels = self.moduler_labels.cuda()
    
#     def create_handler(self, log_dir):
#         file_item=1
#         logging_filename = f"({today}-{file_item}).log"

#         if not os.path.isdir(log_dir):
#             os.mkdir(log_dir)
#             print(f"Created directory: {log_dir}")

#         while os.path.exists(os.path.join(log_dir, logging_filename)):
#             file_item += 1
#             logging_filename = f"{today}-{file_item}.log"

#         print(f"Log File = {log_dir}/{logging_filename}")
    
#         if self.logging.hasHandlers(): self.logging.removeHandler(inf_arl.fileHandler)
#         self.fileHandler=logging.FileHandler("{0}/{1}".format(log_dir, logging_filename))
#         self.fileHandler.setFormatter(logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s" ))
#         self.logging.addHandler(self.fileHandler)

    def load_ssd(self, ssd_path):
        gc.collect()
        torch.cuda.empty_cache()
        label_file = os.path.join('/home/hjchris/data/Pascal_VOC/VOC2007/', "voc-model-labels.txt")
        
        self.class_names = [name.strip() for name in open(label_file).readlines()]
       
        self.log(f"="*45 + f" Inference Session " + f"="*45)
        create_net = create_mobilenetv2_ssd_lite(len(self.class_names), is_test=True)
        
        self.net = create_net.create_SSD()
        self.net.init_from_pretrained_ssd(ssd_path, log=self.log)
        
        self.load_checkpoint(ssd_path)   
        self.predictor = create_mobilenetv2_ssd_lite_predictor(self.net, nms_method='hard', device=DEVICE)
    
    def load_checkpoint(self, ssd_path):
        print("Load checkpoint")
        self.ckpt = torch.load(ssd_path, map_location=lambda storage, loc: storage)
        self.epoch = self.ckpt['epoch']
        self.max_grad_norm = self.ckpt['max_grad_norm']
        self.loss = self.ckpt['loss']
        self.privacy_flag = self.ckpt['privacy_flag']
        self.public_flag = self.ckpt['public_flag']
        self.discriminator_loc = -10
        
#         self.log(f"Epoch={self.epoch}, max_grad norm={self.max_grad_norm}, Loss={self.ckpt['loss']:.4f}")    
        self.log(f"Epoch={self.epoch}, max_grad norm={self.max_grad_norm}, Loss={self.ckpt['loss']:.4f}")
        
        if self.privacy_flag:
            
            self.discriminator_loc = self.ckpt['discriminator_loc'] if not "Attack" in ssd_path else self.ckpt['attacker_loc']
            self.sensitive_classes = self.ckpt['sensitive_classes']
            self.arl_setting = self.ckpt['arl_setting'] if not "Attack" in ssd_path else "attack"
        
            total_loss = self.ckpt['total_loss'] if 'tota_loss' in self.ckpt.keys() else self.ckpt['loss']
            self.log(f"alpha={self.ckpt['alpha'][0]}, Sensitive Classes={self.ckpt['sensitive_classes']}, Total Loss={total_loss:.4f}, "+ 
                    f"Method={self.arl_setting}")
        
        if self.public_flag:
            self.public_classes = self.ckpt['public_classes']
            self.log(f"Public classes={self.public_classes}")
        
        
    def group_annotation_by_class(self, dataset):
            true_case_stat = {}
            all_gt_boxes = {}
            all_difficult_cases = {}
            
            for i in range(len(dataset)):
                image_id, annotation = dataset.get_annotation(i)
                gt_boxes, classes, is_difficult = annotation
                gt_boxes = torch.from_numpy(gt_boxes)
                for i, difficult in enumerate(is_difficult):
                    class_index = int(classes[i])
                    gt_box = gt_boxes[i]
                    if not difficult:
                        true_case_stat[class_index] = true_case_stat.get(class_index, 0) + 1

                    if class_index not in all_gt_boxes:
                        all_gt_boxes[class_index] = {}
                    if image_id not in all_gt_boxes[class_index]:
                        all_gt_boxes[class_index][image_id] = []
                    all_gt_boxes[class_index][image_id].append(gt_box)
                    if class_index not in all_difficult_cases:
                        all_difficult_cases[class_index]={}
                    if image_id not in all_difficult_cases[class_index]:
                        all_difficult_cases[class_index][image_id] = []
                    all_difficult_cases[class_index][image_id].append(difficult)

            for class_index in all_gt_boxes:
                for image_id in all_gt_boxes[class_index]:
                    all_gt_boxes[class_index][image_id] = torch.stack(all_gt_boxes[class_index][image_id])
            for class_index in all_difficult_cases:
                for image_id in all_difficult_cases[class_index]:
                    all_gt_boxes[class_index][image_id] = all_gt_boxes[class_index][image_id].clone().detach()
            return true_case_stat, all_gt_boxes, all_difficult_cases
        
    def compute_average_precision_per_class(self, num_true_cases, gt_boxes, difficult_cases,
                                        prediction_file, iou_threshold, use_2007_metric):
        
         #  Read the files one by one. Each file is one class
        with open(prediction_file) as f:
            image_ids = []
            boxes = []
            scores = []
            
            #  Read the lines. Format: image_id, confidence, 4 coordinates
            for line in f:
                t = line.rstrip().split(" ")
                image_ids.append(t[0])
                scores.append(float(t[1]))
                box = torch.tensor([float(v) for v in t[2:]]).unsqueeze(0)
                box -= 1.0  # convert to python format where indexes start from 0
                boxes.append(box)
                
            scores = np.array(scores)
            
            #  Inverse indexes. Smallest confidence in the front.
            sorted_indexes = np.argsort(-scores)
            boxes = [boxes[i] for i in sorted_indexes]
            image_ids = [image_ids[i] for i in sorted_indexes]
            
            #  len(image_ids) = number of boxes predicted as this class in the whole dataset. 
            true_positive = np.zeros(len(image_ids))
            false_positive = np.zeros(len(image_ids))
            matched = set()
            
             #  Calculate true_positive and false_positive for each box.
            for i, image_id in enumerate(image_ids):
                box = boxes[i]
                
                # if the image is not in the sets --> this prediction is false
                if image_id not in gt_boxes:
                    false_positive[i] = 1
                    continue

                gt_box = gt_boxes[image_id]
                ious = box_utils.iou_of(box, gt_box)
                max_iou = torch.max(ious).item()
                max_arg = torch.argmax(ious).item()
                
                # if the iou is larger enough:
                if max_iou > iou_threshold:
                    
                    # if it is not a difficult case:
                    if difficult_cases[image_id][max_arg] == 0:
                        
                        # if the ground truth box has not been predicted --> true prediction.
                        if (image_id, max_arg) not in matched:
                            true_positive[i] = 1
                            matched.add((image_id, max_arg))
                            
                        # if the the ground truth box is already predicted --> false prediction.
                        else:
                            false_positive[i] = 1
                
                # if the iou is smaller than 0.5 --> consider ths prediction is false.
                else:
                    false_positive[i] = 1

        true_positive = true_positive.cumsum()
        false_positive = false_positive.cumsum()
        precision = true_positive / (true_positive + false_positive)
        recall = true_positive / num_true_cases
        
        # It computes the under curve area of precision and recall. Recall follows the normal definition. Precision is a variant.
        if use_2007_metric:
            return measurements.compute_voc2007_average_precision(precision, recall)
        else:
            return measurements.compute_average_precision(precision, recall)
   
    def write_results(self, eval_path, discriminator_loc):
        timer = Timer()
        results = []
        for i in range(len(self.dataset)):
            timer.start("Load Image")
            image = self.dataset.get_image(i)
            if i%100 == 0: print(f"[{i:4d}]"+ "Load Image: {:4f} seconds.".format(timer.end("Load Image")), end=' ')
            timer.start("Predict")
            boxes, labels, probs = self.predictor.predict(image, i, discriminator_loc)
            if i%100 == 0:print("Prediction: {:4f} seconds.".format(timer.end("Predict")))
            indexes = torch.ones(labels.size(0), 1, dtype=torch.float32) * i
            results.append(torch.cat([
                indexes.reshape(-1, 1),
                labels.reshape(-1, 1).float(),
                probs.reshape(-1, 1),
                boxes + 1.0  # matlab's indexes start from 1
            ], dim=1))
            
        results = torch.cat(results)
        
        for class_index, class_name in enumerate(self.class_names):
            if class_index == 0: 
                continue  # ignore background
            
            prediction_path = eval_path + f"det_test_{class_name}.txt"
            
            with open(prediction_path, "w") as f:
                sub = results[results[:, 1] == class_index, :]
                for i in range(sub.size(0)):
                    prob_box = sub[i, 2:].numpy()
                    image_id = self.dataset.ids[int(sub[i, 0])]
                    print(
                        image_id + " " + " ".join([str(v) for v in prob_box]),
                        file=f
                    )

    def inference_ssd(self, eval_path):  
        ''' =====================================================================
        Inference encoder and decoder.
            1. 
            2. 
        ===================================================================== '''        
        
        self.log(f"Results are stored at {eval_path}")        
       
        # Run results
        count_files = len([name for name in os.listdir(eval_path) if os.path.isfile(os.path.join(eval_path, name))])
        if count_files != 20: 
            print("Write Files.")
            self.write_results(eval_path, self.discriminator_loc)
        
        # Read results
        aps = []
        self.log("Average Precision Per-class:")
            
        for class_index, class_name in enumerate(self.class_names):
            if class_index == 0:
                continue
                
            prediction_path = eval_path + f"det_test_{class_name}.txt"
            ap = self.compute_average_precision_per_class(
                self.true_case_stat[class_index],
                self.all_gb_boxes[class_index],
                self.all_difficult_cases[class_index],
                prediction_path,
                self.iou_threshold,
                self.use_2007_metric
            )
            aps.append(ap)
            self.log(f"{class_name}: {ap}")
                
        self.log(f"Average Precision Across All Classes:{sum(aps)/len(aps)}")
        
    def load_moduler(self, ssd_path, module="discriminator", size=300, mean=0.0, std=1.0, verbose=False):
        self.log(f"Load {module}.")
        
        self.transform = PredictionTransform(size, mean, std)
        create_discriminator = Create_Discriminator()
        
        # need update
        if module == "discriminator":
            self.discriminator_loc = self.ckpt['discriminator_loc'] 
        elif module == "attacker":
            self.discriminator_loc = self.ckpt['attacker_loc']
        elif module == "public": 
            self.discriminator_loc = self.ckpt['discriminator_loc']
        else:
            raise Exception(f"Only discriminator, attacker, or public.")
            
        self.moduler = create_discriminator.build(self.discriminator_loc)
        self.moduler.load(ssd_path, module = module)
            
    def online_sensitive_labels(self, labels, classes=[15]):
        temp = torch.zeros(len(labels), dtype=torch.long)
        for j in range(len(temp)):
            temp[j] = (1 if any(items in labels[j].unique() for items in classes) else 0)   
        return temp
    
    def image_transform(self, image):
        image = self.transform(image)
        image = image.unsqueeze(0)
        image = image.to(self.device)
        return image
    
    def inference_moduler(self, val_loader, module, print_every=5):
        self.log(f"Inference {module}.")
            
        if module == "attacker" or module == "discriminator":
            module_classes = self.ckpt['sensitive_classes']
        elif module == "public":
            module_classes = self.ckpt['public_classes']
        else:
            raise Exception(f"Only attacker, discriminator, or public.")
            
        config = mobilenetv1_ssd_config
        criterion = MultiboxLoss(config.priors, iou_threshold=0.5, neg_pos_ratio=3, center_variance=0.1, size_variance=0.2, device=DEVICE)
        
        self.net.eval()
        self.moduler.eval()
        timer = Timer()
        train_correct = 0.0
        loss = 0.0
        num = 0
        check = 0
        TP, TN, FP, FN = 0, 0, 0, 0
        timer.start("Inference Image")
        itr = 0

        for (i, data) in enumerate(val_loader):
            
            itr += 1

#             self.image = self.image_transform(data[0])
            
#             labels = torch.from_numpy(data[2].reshape(1, -1))        
#             self.labels.resize_(labels.size()).copy_(labels)

            
#             sensitive_labels = data[3] if len(data) == 4 else self.online_sensitive_labels(self.labels, self.ssd_ckpt['sensitive_classes'])
#             self.sensitive_labels.resize_(sensitive_labels.size()).copy_(sensitive_labels)  
            
            self.images.resize_(data[0].size()).copy_(data[0])
            self.labels.resize_(data[2].size()).copy_(data[2])
            sensitive_labels = data[3] if len(data) == 4 else self.online_sensitive_labels(self.labels, module_classes)
            self.moduler_labels.resize_(sensitive_labels.size()).copy_(sensitive_labels)     
            
            with torch.no_grad():
                _, _, z = self.net(self.images, self.discriminator_loc)
                z = self.moduler(z)

                d_prob = F.softmax(z, dim=1)
                loss += self.nll_loss(torch.log(d_prob+1e-16), self.moduler_labels)
            
            probability, indecies = torch.max(d_prob, dim=1)
            train_correct += np.sum(np.array(self.moduler_labels.cpu())==np.array(indecies.cpu()))
            num += indecies.size()[0]

            # True Positive (TP): we predict a label of 1 (positive), and the true label is 1.
            TP += torch.sum(torch.logical_and(torch.unsqueeze(indecies,0) == 1, torch.unsqueeze(self.moduler_labels,0) == 1))

            # True Negative (TN): we predict a label of 0 (negative), and the true label is 0.
            TN += torch.sum(torch.logical_and(torch.unsqueeze(indecies,0) == 0, torch.unsqueeze(self.moduler_labels,0) == 0))

            # False Positive (FP): we predict a label of 1 (positive), but the true label is 0.
            FP += torch.sum(torch.logical_and(torch.unsqueeze(indecies,0) == 1, torch.unsqueeze(self.moduler_labels,0) == 0))

            # False Negative (FN): we predict a label of 0 (negative), but the true label is 1.
            FN += torch.sum(torch.logical_and(torch.unsqueeze(indecies,0) == 0, torch.unsqueeze(self.moduler_labels,0) == 1))

            check += torch.sum(self.moduler_labels == 1) 
            
            if itr % print_every == 0: 
                print(f"[{itr:4d}][{num:4d}] " + "Inference: {:4f}s.".format(timer.end("Inference Image")) + 
                      f"Accuracy = {float(train_correct/num):4f}, Loss = {float(loss/itr):.4f}, " + 
                      f"[TP = {float(TP)/num*100:.2f}%, FN = {float(FN)/num*100:.2f}%, TN = {float(TN)/num*100:.2f}%, FP = {float(FP)/num*100:.2f}%]")
                timer.start("Inference Image")

        timer.end("Inference Image")
        
        self.log(f"{module} Loss = {float(loss/itr):.4f}. {module} Accuracy = {train_correct/num}")
        self.log(f"TP={float(TP)/num*100:.2f}%, FN={float(FN)/num*100:2.2f}%, " +  
                          f"TN={float(TN)/num*100:2.2f}%, FP={float(FP)/num*100:2.2f}%, Total Images={num}, 1={check}.")

# Inference 

In [ ]:
def create_Handler(log, log_dir, path_name):
    file_item=1
    logging_filename = f"{path_name} ({today}-{file_item}).log"

    if not os.path.isdir(log_dir):
        os.mkdir(log_dir)
        print(f"Created directory: {log_dir}")

    while os.path.exists(os.path.join(log_dir, logging_filename)):
        file_item += 1
        logging_filename = f"{path_name} ({today}-{file_item}).log"

    print(f"Log File = {log_dir}/{logging_filename}")

    if log.hasHandlers(): 
        log.removeHandler(inf_fileHandler)
        log.removeHandler(streamHandler)
    
    fileHandler = logging.FileHandler("{0}/{1}".format(log_dir, logging_filename))
    fileHandler.setFormatter(logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s" ))
    
    log.addHandler(fileHandler)
    log.addHandler(streamHandler)
    
    return fileHandler

## run

In [ ]:
inference_date = 1229
exp_nums = [1, 2, 3, 4, 5, 6, 7, 8]


for exp_num in exp_nums:
    ssd_path, eval_path = find_ssd_file('/home/hjchris/research/mitigating/chris/pascal_ckpts/' + f"{inference_date}_21/{str(inference_date)}_21-{str(exp_num)}/")

    to_log = True
    if to_log: inf_fileHandler = create_Handler(inf_log, log_dir, ssd_path[61:70])
    inf_arl = Inference_ARL(to_log=to_log, log=inf_log.info)
    inf_arl.load_ssd(ssd_path)
    inf_arl.inference_ssd(eval_path)

    data_path = f"/home/hjchris/data"
    data_prep = Data_Preprocessing()
    data_prep.load_online(data_path)

    if "Attack" in ssd_path:
        inf_arl.load_moduler(ssd_path, module="attacker")
        inf_arl.inference_moduler(data_prep.val_loader, module="attacker")
    elif "FM" in ssd_path:
        if inf_arl.privacy_flag:
            #module = "discriminator" #public or discriminator
            inf_arl.load_moduler(ssd_path, module="discriminator")
            inf_arl.inference_moduler(data_prep.val_loader, module="discriminator")
        if inf_arl.public_flag:
            inf_arl.load_moduler(ssd_path, module="public")
            inf_arl.inference_moduler(data_prep.val_loader, module="public")

The reason that average precision is nan is that when the difficult cases == 1.

`difficult': an object marked as `difficult' indicates that the object is considered difficult to recognize, for example an object which is clearly visible but unidentifiable without substantial use of context. Objects marked as **difficult are currently ignored** in the evaluation of the challenge.

Average Precision Per-class:

aeroplane: 0.6973327307871002

bicycle: 0.7823755921687233

bird: 0.6342429230125619

boat: 0.5478160937380846

bottle: 0.3564069147093762

bus: 0.7882037885117419

car: 0.7444122242934775

cat: 0.8198865557991936

chair: 0.5378973422880109

cow: 0.6186076149254742

diningtable: 0.7369559500950861

dog: 0.7848265495754562

horse: 0.8222948787839229

motorbike: 0.8057808854619948

person: 0.7176976451996411

pottedplant: 0.42802932547480066

sheep: 0.6259124005994047

sofa: 0.7840368059271103

train: 0.8331588002612781

tvmonitor: 0.6555051795079904

Average Precision Across All Classes:0.6860690100560214

In [ ]:
ssd_path = f"/home/hjchris/research/mitigating/chris/pascal_ckpts/1229_21/1229_21-8/mb2-ssd-lite-Attack-Ep200-FM21-Loss-2.8964+0.3243-Ent-0.3584.pth"
ckpt = torch.load(ssd_path, map_location=lambda storage, loc: storage)

In [ ]:
ckpt['attacked_model']